In [2]:
import sys
sys.path.append("python/")

import sympy as sp
import numdifftools as nd
import numpy as np
import scipy.optimize as opt

import datlib
import symbase
import Z2sym
import Z3sym
import Z3num

# Ферромагнетик в симметричном случае

## Выражение для $Z^{(3)}$

### Аргументы

In [3]:
hi, hj, lij, lik = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik 
s1, s2, s3 = Z3sym.s1, Z3sym.s2, Z3sym.s3
x, t = sp.symbols("x t")

In [4]:
arg_exp = hj*x
arg_exp = arg_exp.replace(
    x, sp.solve(sp.sqrt(hi**2+lij**2+2*hi*lij*x)-t, x)[0]
).expand()
arg_exp += (s1+s2)*t + s1*s2*lik
expr_limit = hi+s3*lij
arg_F2 = (t+(s1+s2)*lik)/sp.sqrt(2*lik)

arg_exp = arg_exp.replace(t, t-(s1+s2)*lij*hi/hj).expand()
arg_F2 = arg_F2.replace(t, t-(s1+s2)*lij*hi/hj).expand()
expr_limit += (s1+s2)*lij*hi/hj

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*lij*hi/hj)).expand()
arg_F2 = arg_F2.replace(t, t*sp.sqrt(2*lij*hi/hj)).expand()
expr_limit /= sp.sqrt(2*lij*hi/hj)

display(arg_exp)
display(arg_F2)
display(expr_limit)

-\sigma_1*\sigma_2*\lambda_{ij}*h_i/h_j + \sigma_1*\sigma_2*\lambda_{ik} - \lambda_{ij}*h_i/h_j - \lambda_{ij}*h_j/(2*h_i) + t**2 - h_i*h_j/(2*\lambda_{ij})

-sqrt(2)*\sigma_1*\lambda_{ij}*h_i/(2*sqrt(\lambda_{ik})*h_j) + sqrt(2)*\sigma_1*sqrt(\lambda_{ik})/2 - sqrt(2)*\sigma_2*\lambda_{ij}*h_i/(2*sqrt(\lambda_{ik})*h_j) + sqrt(2)*\sigma_2*sqrt(\lambda_{ik})/2 + sqrt(\lambda_{ij})*sqrt(h_i)*t/(sqrt(\lambda_{ik})*sqrt(h_j))

sqrt(2)*sqrt(h_j)*(\sigma_3*\lambda_{ij} + \lambda_{ij}*h_i*(\sigma_1 + \sigma_2)/h_j + h_i)/(2*sqrt(\lambda_{ij})*sqrt(h_i))

In [5]:
arg_exp -= t**2
arg_exp += expr_limit**2
arg_exp = arg_exp.expand()
display(arg_exp)

\sigma_1*\sigma_2*\lambda_{ik} + \sigma_1*\sigma_3*\lambda_{ij} + \sigma_1*h_i + \sigma_2*\sigma_3*\lambda_{ij} + \sigma_2*h_i + \sigma_3*h_j

### Производные $F^{(3)}$

In [6]:
t, x, a, b = sp.symbols("t x a b")

In [7]:
Z2sym.F2(a*t+b).diff(t).expand()

-2*a**2*t*F2(a*t + b) - 2*a*b*F2(a*t + b) + a

In [8]:
def get_I2_n(x, n):
    """I^{(2)}_n(x)"""
    _expr_n_0 = 1/sp.sqrt(a)*Z2sym.F2(sp.sqrt(a)*x)*sp.exp(x**2*(a-1))
    _expr_n_1 = 1/(2*a)*sp.exp(-x**2)*(sp.exp(x**2*a)-1)

    k = n // 2
    if n % 2 == 0:
        return _expr_n_0.diff(a, k).replace(a, 1).expand()
    else:
        return _expr_n_1.diff(a, k).replace(a, 1).expand()

In [9]:
get_I2_n(x, 4)

x**3/2 - 3*x/4 + 3*F2(x)/4

In [10]:
class F3_dummy(sp.Function):
    def _latex(self, printer, exp=None):
        arg1, arg2, arg3 = list(map(printer.doprint, self.args))
        return "F_{dummy}^{(3)}"+f"\\left({arg1} \, | \, {arg2}, {arg3} \\right)" 

In [11]:
def get_I3_n(x, a, b, n):
    """I^{(3)}_n(x, a, b)"""
    if a == 1:
        if b == 0:
            result = x**(n+1)*Z2sym.F2(x)
            result -= get_I2_n(x, n+1)
            return result/(n+1)

        # b != 0
        result = -x**n*Z2sym.F2(x+b)
        result += get_I2_n(x, n)

        if n == 0:
            result += sp.exp(-x**2)*Z2sym.F2(b)
        if n != 0:
            result += n*get_I3_n(x, a, b, n-1)

        return result/(2*b)

    # a != 1, b != 0
    if n == 0:
        return F3_dummy(x, a, b)

    result = x**(n-1)*Z2sym.F2(a*x+b)
    result -= a*get_I2_n(x, n-1)
    result += 2*a*b*get_I3_n(x, a, b, n-1)

    if n == 1:
        result -= sp.exp(-x**2)*Z2sym.F2(b)
    if n != 1:
        result -= (n-1)*get_I3_n(x, a, b, n-2)
    
    return result/(2*(1-a**2))

In [12]:
get_I3_n(x, a, b, 1).expand()

2*a*b*F3_dummy(x, a, b)/(2 - 2*a**2) - a*F2(x)/(2 - 2*a**2) - F2(b)/(-2*a**2*exp(x**2) + 2*exp(x**2)) + F2(a*x + b)/(2 - 2*a**2)

In [13]:
class F3(sp.Function):
    def _latex(self, printer, exp=None):
        arg1, arg2, arg3 = list(map(printer.doprint, self.args))
        return "F^{(3)}"+f"\\left({arg1} \, | \, {arg2}, {arg3} \\right)" 

    def fdiff(self, argindex=1):
        if argindex == 1:
            x, a, b = self.args
            return Z2sym.F2(a*x+b)-2*x*F3(*self.args)
        
        t, x, a, b = sp.symbols("t x a b")
        expr = Z2sym.F2(a*t+b).diff([a, b][argindex-2]).expand()
        expr = sp.collect(expr, [Z2sym.F2(a*t+b)], evaluate=False)

        I3 = expr[Z2sym.F2(a*t+b)]
        I2 = expr[1]

        expr = 0
        I3 = sp.Poly(I3, t).as_dict()
        for key, item in I3.items():
            expr += item*get_I3_n(x, a, b, key[0])
        I2 = sp.Poly(I2, t).as_dict()
        for key, item in I2.items():
            expr += item*get_I2_n(x, key[0])

        subs = dict(zip([x, a, b], self.args))
        subs.update({F3_dummy(x, a, b): F3(*self.args)})
        return expr.subs(subs) 

In [14]:
F3(x, a, b).diff(b)

-2*a*(2*a*b*F3(x, a, b) - a*F2(x) + F2(a*x + b) - exp(-x**2)*F2(b))/(2 - 2*a**2) - 2*b*F3(x, a, b) + F2(x)

In [15]:
class F3_degenerate(sp.Function):
    def _latex(self, printer, exp=None):
        arg1, arg2, arg3, arg4, arg5 = list(map(printer.doprint, self.args))
        return "F_{deg,"+f"{arg4},{arg5}"+"}^{(3)}"+f"\\left({arg1} \, | \, {arg2}, {arg3} \\right)" 
    
    def fdiff(self, argindex=1):
        if argindex == 1:
            x, a, b = sp.symbols("x a b")
            subs = dict(zip([x, a, b], self.args[:3]))
            expr = Z2sym.F2(a*x+b).diff(a, self.args[3]).diff(b, self.args[4])

            x, a, b, _, _ = self.args
            return expr.subs(subs) -2*x*F3_degenerate(*self.args)
        
        args = list(self.args)
        args[argindex+1] += 1
        return F3_degenerate(*args)
    

def evaluate_F3_degenerate(expr):
    def evaluate(*args):
        t, x, a, b = sp.symbols("t x a b")
        expr = Z2sym.F2(a*t+b).diff(a, args[3])
        expr = expr.diff(b, args[4]).expand()
        expr = sp.collect(expr, [Z2sym.F2(a*t+b)], evaluate=False)
        
        I3 = expr.get(Z2sym.F2(a*t+b), sp.Integer(0)).subs({a: 1, b: 0})
        I2 = expr.get(1, sp.Integer(0)).subs({a: 1, b: 0})

        expr = 0
        I3 = sp.Poly(I3, t).as_dict()
        for key, item in I3.items():
            expr += item*get_I3_n(x, 1, 0, key[0])
        I2 = sp.Poly(I2, t).as_dict()
        for key, item in I2.items():
            expr += item*get_I2_n(x, key[0])

        subs = dict(zip([x, a, b], args[:3]))
        return expr.subs(subs) 
    
    return expr.replace(F3_degenerate, evaluate)

In [16]:
expr = F3_degenerate(x, a, b, 0, 0).diff(x, a)
expr = evaluate_F3_degenerate(expr)

expr

-x*(-4*x**3*F2(x)/3 + 2*x**2/3 + 2*(a*x + b)*F2(a*x + b) - 2/3 - exp(-x**2)/3)

### Производные $\cal F^{(3)}$

In [17]:
arg_x = (hi+s3*lij+(s1+s2)*hi/hj*lij)/sp.sqrt(2*hi/hj*lij)
arg_a = sp.sqrt(hi/hj*lij)/sp.sqrt(lik)
arg_b = (s1+s2)/sp.sqrt(2*lik)*(lik-hi/hj*lij)
arg_expr = s3*hj+s1*s2*lik+(s1+s2)*(hi+s3*lij)

In [18]:
expr = arg_x*arg_a+arg_b
expr.factor()

sqrt(2)*(\sigma_1*\lambda_{ik} + \sigma_2*\lambda_{ik} + \sigma_3*\lambda_{ij} + h_i)/(2*sqrt(\lambda_{ik}))

In [19]:
expr = arg_expr-arg_x**2
expr = expr.expand()
expr

-\sigma_1*\sigma_2*\lambda_{ij}*h_i/h_j + \sigma_1*\sigma_2*\lambda_{ik} - \lambda_{ij}*h_i/h_j - \lambda_{ij}*h_j/(2*h_i) - h_i*h_j/(2*\lambda_{ij})

In [20]:
_sigmas_expr_E = sp.exp(arg_expr).expand()
_sigmas_expr_F = Z2sym.F2(arg_x).expand()
_sigmas_expr_FTilde = Z2sym.F2(arg_x*arg_a+arg_b).expand()
_sigmas_expr_cF3 = F3(arg_x, arg_a, arg_b).expand()

_sigmas_terms = {
    _sigmas_expr_E*_sigmas_expr_cF3: sp.Dummy(),
    _sigmas_expr_E*_sigmas_expr_F: sp.Dummy(),
    _sigmas_expr_E*_sigmas_expr_FTilde: sp.Dummy(),
    _sigmas_expr_E: sp.Dummy()
}

In [21]:
class Z3SigmasFunctionBase(sp.Function):
    def __init__(self, name, *args, **kwargs):
        self.name = name

    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:4]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}, {args[3]}\\right)"

        indexes = ""
        for i, item in enumerate(self.args[4:]):
            if item:
                indexes += str(i+1)
        name = f"{self.name}"+"_{" + indexes + "}"

        if exp is None:
            return name+args
        else:
            exp = printer.doprint(exp)
            return f"{name}^{exp}"+args

    def get_sigmas_factor(self):
        factor = 1
        if self.args[4]:
            factor *= s1
        if self.args[5]:
            factor *= s2
        if self.args[6]:
            factor *= s3
        return factor

In [22]:
class EFunction(Z3SigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__("{\\cal E}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sigmas_expr_E).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])

        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        for key, item in terms[_sigmas_expr_E].items():
            result += item.subs(subs)*EFunction(*self.args[:4], *key)

        return result

    @classmethod
    def eval(cls, *args):
        if (not args[4]) and args[5]:
            return EFunction(*args[:4], 1, 0, args[-1])

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = self.get_sigmas_factor()*_sigmas_expr_E
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [23]:
display(EFunction(hi, hj, lij, lik, 1, 1, 1).diff(hi).rewrite(sp.exp)-EFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp).diff(hi))

0

In [24]:
class FFunction(Z3SigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__("{\\cal F}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sigmas_expr_E*_sigmas_expr_F).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])

        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        if _sigmas_expr_E in terms:
            for key, item in terms[_sigmas_expr_E].items():
                result += item.subs(subs)*EFunction(*self.args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_F].items():
            result += item.subs(subs)*FFunction(*self.args[:4], *key)

        return result

    @classmethod
    def eval(cls, *args):
        if (not args[4]) and args[5]:
            return FFunction(*args[:4], 1, 0, args[-1])

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = self.get_sigmas_factor()*_sigmas_expr_E*_sigmas_expr_F
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [25]:
display(FFunction(hi, hj, lij, lik, 1, 1, 1).diff(lik).rewrite(sp.exp)-FFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp).diff(lik))

0

In [26]:
class FTildeFunction(Z3SigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__( "\\tilde{\\cal F}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sigmas_expr_E*_sigmas_expr_FTilde).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])

        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        if _sigmas_expr_E in terms:
            for key, item in terms[_sigmas_expr_E].items():
                result += item.subs(subs)*EFunction(*self.args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_FTilde].items():
            result += item.subs(subs)*FTildeFunction(*self.args[:4], *key)

        return result

    @classmethod
    def eval(cls, *args):
        if (not args[4]) and args[5]:
            return FTildeFunction(*args[:4], 1, 0, args[-1])

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = self.get_sigmas_factor()*_sigmas_expr_E*_sigmas_expr_FTilde
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [27]:
expr = FTildeFunction(hi, hj, lij, lik, 1, 1, 1).diff(hi).rewrite(sp.exp)
expr -= FTildeFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp).diff(hi)
expr = expr.factor()
expr

0

In [28]:
class cF3(sp.Function):
    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:4]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}, {args[3]}\\right)"
        if exp is None:
            return "{\\cal F}^{(3)}"+args
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"
    
    def fdiff(self, argindex=1):
        expr = (s1*s2*s3*_sigmas_expr_E*_sigmas_expr_cF3).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])
        
        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        for key, item in terms[_sigmas_expr_E].items():
            result += item.factor().subs(subs)*EFunction(*self.args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_F].items():
            result += item.factor().subs(subs)*FFunction(*self.args[:4], *key)
        
        for key, item in terms[_sigmas_expr_E*_sigmas_expr_FTilde].items():
            result += item.factor().subs(subs)*FTildeFunction(*self.args[:4], *key)
        
        for key, item in terms[_sigmas_expr_E*_sigmas_expr_cF3].items():
            if key != (1, 1, 1):
                raise RuntimeError
            result += item.factor().subs(subs)*cF3(*self.args)

        return result

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = s1*s2*s3*_sigmas_expr_E*_sigmas_expr_cF3
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [29]:
sp.Add(*map(lambda x: x*4*lij*hi**2*sp.sqrt(hj)*(lij*hi-lik*hj), cF3(hi, hj, lij, lik).diff(hi).args))

-\lambda_{ij}**(3/2)*sqrt(\lambda_{ik})*h_i**(3/2)*h_j*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1) + sqrt(2)*\lambda_{ij}**(3/2)*\lambda_{ik}*sqrt(h_i)*h_j**2*FTildeFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 0) + sqrt(2)*sqrt(\lambda_{ij})*h_i**(3/2)*h_j*(2*\lambda_{ij}*h_i - \lambda_{ik}*h_j)*FTildeFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1) + 2*sqrt(2)*\lambda_{ij}**2*sqrt(\lambda_{ik})*h_i**2*sqrt(h_j)*FFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 0, 1) + 2*sqrt(h_j)*(\lambda_{ij}**3*h_i*h_j - \lambda_{ij}**2*\lambda_{ik}*h_j**2 - \lambda_{ij}**2*h_i**2 - \lambda_{ij}*h_i**3*h_j + \lambda_{ik}*h_i**2*h_j**2)*cF3(h_i, h_j, \lambda_{ij}, \lambda_{ik})

In [30]:
class cF3_degenerate(sp.Function):
    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args))
        value_args = f"\\left({args[0]}, {args[1]}, {args[2]}, {args[3]}\\right)"
        diffs_args = f"{args[4]}, {args[5]}, {args[6]}, {args[7]}"
        if exp is None:
            return "{\\cal F}_{deg,"+diffs_args+"}"+value_args
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"
    
    def fdiff(self, argindex=1):    
        args = list(self.args)
        args[argindex+3] += 1
        return cF3_degenerate(*args)

def evaluate_cF3_degenerate(expr):
    def evaluate(*args):
        expr = (s1*s2*s3*_sigmas_expr_E*F3_degenerate(arg_x, arg_a, arg_b, 0, 0)).diff(hi, args[4])
        expr = expr.diff(hj, args[5]).diff(lij, args[6]).diff(lik, args[7])
        
        expr = evaluate_F3_degenerate(expr)

        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])
        
        subs = dict(zip([hi, hj, lij, lik], args[:4]))
        
        result = 0
        for key, item in terms[_sigmas_expr_E].items():
            result += item.factor().subs(subs)*EFunction(*args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_F].items():
            result += item.factor().subs(subs)*FFunction(*args[:4], *key)

        if _sigmas_expr_E*_sigmas_expr_FTilde in terms:
            for key, item in terms[_sigmas_expr_E*_sigmas_expr_FTilde].items():
                result += item.factor().subs(subs)*FFunction(*args[:4], *key)

        return result
    
    return expr.replace(cF3_degenerate, evaluate)

In [31]:
# expr = cF3_degenerate(hi, hj, lij, lik, *(4*[0])).diff(lij, 1)
# expr = evaluate_cF3_degenerate(expr)

# expr

### Расчет $F^{(2)}$ и $F^{(3)}$

#### Определения

In [32]:
class erfcx(sp.Function):
    def _latex(self, printer, exp=None):
        arg = printer.doprint(self.args[0])
        if exp is None:
            return "\operatorname{erfcx}"+f"\\left({arg}\\right)"
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"

In [33]:
def erfi_to_dawsn(expr):
    def __helper(z):
        return Z2sym.F2(z)*2/sp.sqrt(sp.pi)*sp.exp(z**2)
    return expr.replace(sp.erfi, __helper)


def erfc_to_erfcx(expr):
    def __helper(z):
        return erfcx(z)*sp.exp(-z**2)
    return expr.replace(sp.erfc, __helper)

In [34]:
t, x, a, b = sp.symbols("t x a b")
n, h = sp.symbols("n h")
n0 = sp.Symbol("n_0")

#### $c > 0$

In [35]:
c = sp.Symbol("c", positive=True)
expr = t**2-(a*t+b-n*h)**2
expr = expr.expand()
expr = expr.replace(a**2, 1-c).expand()
expr = sp.integrate(sp.exp(expr), [t, 0, x])*sp.exp(-x**2)/(n*sp.sqrt(sp.pi))
expr = expr.expand()
expr = erfi_to_dawsn(expr)
expr = sp.Add(*map(lambda x: x.simplify(), expr.args))
expr

exp(-b**2 + 2*b*h*n - h**2*n**2 - x**2)*F2(a*(b - h*n)/sqrt(c))/(sqrt(pi)*sqrt(c)*n) + exp(-2*a*b*x + 2*a*h*n*x - b**2 + 2*b*h*n + c*x**2 - h**2*n**2 - x**2)*F2((-a*b + a*h*n + c*x)/sqrt(c))/(sqrt(pi)*sqrt(c)*n)

In [36]:
print(sp.solve(expr.args[0].args[-1].diff(n), n)[0])

(a*x + b)/h


In [37]:
print(sp.solve(expr.args[1].args[-1].diff(n), n)[0])

b/h


In [38]:
expr = -2*a*b*x + 2*a*h*n*x - b**2 + 2*b*h*n + c*x**2 - h**2*n**2 - x**2
expr = expr.replace(n, n0+n).expand()
expr = expr.collect(n)
sp.print_latex(expr)
expr

- 2 a b x + 2 a h n_{0} x - b^{2} + 2 b h n_{0} + c x^{2} - h^{2} n^{2} - h^{2} n_{0}^{2} + n \left(2 a h x + 2 b h - 2 h^{2} n_{0}\right) - x^{2}


-2*a*b*x + 2*a*h*n_0*x - b**2 + 2*b*h*n_0 + c*x**2 - h**2*n**2 - h**2*n_0**2 + n*(2*a*h*x + 2*b*h - 2*h**2*n_0) - x**2

In [39]:
expr = -b**2 + 2*b*h*n - h**2*n**2 - x**2
expr = expr.replace(n, n0+n).expand()
expr = expr.collect(n)
sp.print_latex(expr)
expr

- b^{2} + 2 b h n_{0} - h^{2} n^{2} - h^{2} n_{0}^{2} + n \left(2 b h - 2 h^{2} n_{0}\right) - x^{2}


-b**2 + 2*b*h*n_0 - h**2*n**2 - h**2*n_0**2 + n*(2*b*h - 2*h**2*n_0) - x**2

#### $c < 0$

In [40]:
c = sp.Symbol("c", negative=True)
expr = t**2-(a*t+b-n*h)**2
expr = expr.expand()
expr = expr.replace(a**2, 1-c).expand()
expr = sp.integrate(sp.exp(expr), [t, 0, x])*sp.exp(-x**2)/(n*sp.sqrt(sp.pi))
expr = expr.simplify()
sp.print_latex(expr)
display(expr)

expr = expr.rewrite(sp.erfc)
expr = expr.expand().factor().simplify()
display(expr)
expr = erfc_to_erfcx(expr).expand()
expr = sp.Add(*map(lambda x: x.simplify(), expr.args))
sp.print_latex(expr)
display(expr)

\frac{\left(\operatorname{erf}{\left(\frac{a b - a h n - c x}{\sqrt{- c}} \right)} - \operatorname{erf}{\left(\frac{a \left(b - h n\right)}{\sqrt{- c}} \right)}\right) e^{\frac{- a^{2} b^{2} - a^{2} h^{2} n^{2} - b^{2} c + 2 b h n \left(a^{2} + c\right) - c h^{2} n^{2} - c x^{2}}{c}}}{2 n \sqrt{- c}}


(erf((a*b - a*h*n - c*x)/sqrt(-c)) - erf(a*(b - h*n)/sqrt(-c)))*exp((-a**2*b**2 - a**2*h**2*n**2 - b**2*c + 2*b*h*n*(a**2 + c) - c*h**2*n**2 - c*x**2)/c)/(2*n*sqrt(-c))

(erfc((-a*b + a*h*n + c*x)/sqrt(-c)) + erfc(a*(b - h*n)/sqrt(-c)) - 2)*exp(-a**2*b**2/c + 2*a**2*b*h*n/c - a**2*h**2*n**2/c - b**2 + 2*b*h*n - h**2*n**2 - x**2)/(2*n*sqrt(-c))

\frac{\operatorname{erfcx}\left(\frac{- a b + a h n + c x}{\sqrt{- c}}\right) e^{- 2 a b x + 2 a h n x - b^{2} + 2 b h n + c x^{2} - h^{2} n^{2} - x^{2}}}{2 n \sqrt{- c}} + \frac{\operatorname{erfcx}\left(\frac{a \left(b - h n\right)}{\sqrt{- c}}\right) e^{- b^{2} + 2 b h n - h^{2} n^{2} - x^{2}}}{2 n \sqrt{- c}} - \frac{e^{- \frac{a^{2} b^{2}}{c} + \frac{2 a^{2} b h n}{c} - \frac{a^{2} h^{2} n^{2}}{c} - b^{2} + 2 b h n - h^{2} n^{2} - x^{2}}}{n \sqrt{- c}}


erfcx((-a*b + a*h*n + c*x)/sqrt(-c))*exp(-2*a*b*x + 2*a*h*n*x - b**2 + 2*b*h*n + c*x**2 - h**2*n**2 - x**2)/(2*n*sqrt(-c)) + erfcx(a*(b - h*n)/sqrt(-c))*exp(-b**2 + 2*b*h*n - h**2*n**2 - x**2)/(2*n*sqrt(-c)) - exp(-a**2*b**2/c + 2*a**2*b*h*n/c - a**2*h**2*n**2/c - b**2 + 2*b*h*n - h**2*n**2 - x**2)/(n*sqrt(-c))

In [41]:
print(expr.args[0].args[-1].args[0].simplify())

(-a**2*b**2 + 2*a**2*b*h*n - a**2*h**2*n**2 + c*(-b**2 + 2*b*h*n - h**2*n**2 - x**2))/c


In [42]:
expr = sp.exp(-a**2*b**2/c + 2*a**2*b*h*n/c - a**2*h**2*n**2/c - b**2 + 2*b*h*n - h**2*n**2 - x**2)
expr = expr.diff(n)
display(expr)
print(sp.solve(expr, n))

print(sp.expand(expr.args[0]/(2*h)))

(2*a**2*b*h/c - 2*a**2*h**2*n/c + 2*b*h - 2*h**2*n)*exp(-a**2*b**2/c + 2*a**2*b*h*n/c - a**2*h**2*n**2/c - b**2 + 2*b*h*n - h**2*n**2 - x**2)

[b/h]
a**2*b/c - a**2*h*n/c + b - h*n


### Численная проверка

In [43]:
hi, hj, lij, lik = 1.1, 1.4, 0.9, 1.2

print(Z3num.calc_Z3_integrate(hi, hj, lij, lik))
print(2*(2*np.pi)**3/np.sqrt(hi*hj*lij*lik)*Z3num.cF3_symmetrical(hi, hj, lij, lik))

10617.721695485003
10617.722628898244


In [44]:
def get_diffs(expr, args, n=1, func=None):
    diffs = []
    for arg in args:
        _expr = expr.diff(arg, n)
        if func is not None:
            _expr = func(_expr)
        diffs.append(sp.lambdify(args, _expr, Z3sym.num_module))
    return diffs

In [45]:
def calc_numerical_diffs(func, args, n=1):
    diffs = np.zeros_like(args)
    for i, arg in enumerate(args):
        direction = [0] * len(args)
        direction[i] = 1
        diffs[i] = nd.directionaldiff(func, args, direction, n=n)
    return diffs


def calc_exprs(exprs, args):
    return list(map(lambda expr: expr(*args), exprs))


def compare_exprs(exprs_1, exprs_2):
    for i, expr in enumerate(exprs_1):
        print(expr)
        print(exprs_2[i])
        print()

In [46]:
x, a, b = sp.symbols("x a b")

f_F3_diffs = get_diffs(F3(x, a, b), [x, a, b])

args = 1.4, -6, 3
compare_exprs(
    calc_numerical_diffs(lambda args: Z3num.F3(*args), args),
    calc_exprs(f_F3_diffs, args)
)

0.035252886520250576
0.0352528865282183

-0.008072403402538892
-0.008072403578818818

-0.000507731536633192
-0.0005077325358310847



In [47]:
f_F3_degenerate_diffs = get_diffs(F3_degenerate(x, a, b, 0, 0), [x, a, b], func=evaluate_F3_degenerate)

args = 1.4, 1, 0
compare_exprs(
    calc_numerical_diffs(lambda args: Z3num.F3(*args), args),
    calc_exprs(f_F3_degenerate_diffs, args)
)

-0.13020305320591144
-0.13020305307846142

-0.03858106485080844
-0.03858039157748265

0.0335001563309486
0.03349936810550083



In [48]:
expr = F3_degenerate(x, a, b, 0, 0).diff(x, x, a)
expr = evaluate_F3_degenerate(expr)
f_expr = sp.lambdify([x, a, b], expr, Z3sym.num_module)

args = 1.4, 1, 0
f = lambda args: nd.directionaldiff(lambda args: Z3num.F3(*args), args, [1, 0, 0], n=2)
print(nd.directionaldiff(f, args, [0, 1, 0], n=1))
print(f_expr(*args))

0.3994941283222833
0.3994975904058924


In [49]:
hi, hj, lij, lik = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik

f_E_123 = sp.lambdify((hi, hj, lij, lik ), EFunction(hi, hj, lij, lik , 1, 1, 1), Z3sym.num_module)
f_E_123_diffs = get_diffs(EFunction(hi, hj, lij, lik , 1, 1, 1), (hi, hj, lij, lik ))

args = 1.1, 1.6, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(lambda x: f_E_123(*x), args),
    calc_exprs(f_E_123_diffs, args)
)

1793.9908528052315
1793.9908528052451

896.4925453059238
896.4925453059141

1796.1935369487594
1796.1935369487765

899.5611861682389
899.5611861682447



In [50]:
f_expr = sp.lambdify((hi, hj, lij, lik), EFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.2
print(f_expr(*args))
print(f_E_123(*args))

1207.3625478377833
1207.3625478377833


In [51]:
f_F_123 = sp.lambdify((hi, hj, lij, lik), FFunction(hi, hj, lij, lik, 1, 1, 1), Z3sym.num_module)
f_F_123_diffs = get_diffs(FFunction(hi, hj, lij, lik, 1, 1, 1), (hi, hj, lij, lik))

args = 1.0, 1.6, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(lambda x: f_F_123(*x), args),
    calc_exprs(f_F_123_diffs, args)
)

242.9620849610564
242.96192895125

125.49175428921544
125.49169241482082

247.5682653467863
247.5681198639258

140.245898560701
140.24589856068548



In [52]:
f_expr = sp.lambdify((hi, hj, lij, lik), FFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.5
print(f_expr(*args))
print(f_F_123(*args))

293.0673061645827
293.0673061645829


In [53]:
f_F_tilde_123 = sp.lambdify((hi, hj, lij, lik), FTildeFunction(hi, hj, lij, lik, 1, 1, 1), Z3sym.num_module)
f_F_tilde_123_diffs = get_diffs(FTildeFunction(hi, hj, lij, lik, 1, 1, 1), (hi, hj, lij, lik))

args = 1.1, 1.6, 0.9, 1.3
compare_exprs(
    calc_numerical_diffs(lambda x: f_F_tilde_123(*x), args),
    calc_exprs(f_F_tilde_123_diffs, args)
)

326.3793182298885
326.3790925223483

186.48751220440235
186.48751220440238

328.1762328880003
328.1760076009938

176.97507990402082
176.97502735302322



In [54]:
f_expr = sp.lambdify((hi, hj, lij, lik), FTildeFunction(hi, hj, lij, lik, 1, 1, 1), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.9
print(f_expr(*args))
print(f_F_tilde_123(*args))

437.60638894794687
437.60638894794687


In [55]:
f_cF3_diffs = get_diffs(cF3(hi, hj, lij, lik), (hi, hj, lij, lik))

args = 1.1, 1.6, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(lambda x: Z3num.cF3_symmetrical(*x), args),
    calc_exprs(f_cF3_diffs, args)
)

50.18660648620504
50.18650313349194

29.615665884710126
29.6157570638889

51.5874549242983
51.58732212835459

31.11306651669058
31.11318871617283



In [56]:
f_expr = sp.lambdify((hi, hj, lij, lik),cF3(hi, hj, lij, lik).rewrite(sp.exp), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.6
print(f_expr(*args))
print(Z3num.cF3_symmetrical(*args))

62.3891828960169
62.38918289601689


In [57]:
f_cF3_degenerate_diffs = get_diffs(cF3_degenerate(hi, hj, lij, lik, *(4*[0])), (hi, hj, lij, lik), n=1, func=evaluate_cF3_degenerate)

args = 1.1, 1.9, 1.9, 1.1
compare_exprs(
    calc_numerical_diffs(lambda x: Z3num.cF3_symmetrical(*x), args, n=1),
    calc_exprs(f_cF3_degenerate_diffs, args)
)

316.47261030616926
316.472129630341

178.63289692163744
178.63292036916692

307.16676683950595
307.16671544714336

201.50338841492476
201.5034912030162



## Моменты $\cal Z^{(3)}$

### Определения

In [58]:
def get_line_Matrix(expr, like_terms):
    terms = sp.collect(expr, like_terms, evaluate=False)
    line = []
    for term in like_terms + [1]:
        if term in terms:
            if term == 1:
                line.append(-terms[term].factor())
            else:
                line.append(terms[term].factor())
        else:
            line.append(0)
    return line


### Общий случай

#### Символьные вычисления

In [59]:
mi, mj, eta_ij, eta_ik = Z3sym.mi, Z3sym.mj, Z3sym.eta_ij, Z3sym.eta_ik

In [60]:
Z3 = cF3(hi, hj, lij, lik)/sp.sqrt(hi*hj*lij*lik)

expr_mi = sp.log(Z3).diff(hi)/2
expr_mi = sp.Add(*map(sp.factor, expr_mi.expand().args))

expr_mj = sp.log(Z3).diff(hj)
expr_mj = sp.Add(*map(sp.factor, expr_mj.expand().args))

expr_eta_ij = sp.log(Z3).diff(lij)/2
expr_eta_ij = sp.Add(*map(sp.factor, expr_eta_ij.expand().args))

expr_eta_ik = sp.log(Z3).diff(lik)
expr_eta_ik = sp.Add(*map(sp.factor, expr_eta_ik.expand().args))

In [61]:
n, d = sp.fraction(expr_mi.factor())
_expr_mi = n-d*mi

n, d = sp.fraction(expr_mj.factor())
_expr_mj = n-d*mj

n, d = sp.fraction(expr_eta_ij.factor())
_expr_eta_ij = n-d*eta_ij

n, d = sp.fraction(expr_eta_ik.factor())
_expr_eta_ik = n-d*eta_ik

In [62]:
vector_system_moments = [
    cF3(hi, hj, lij, lik),
    FTildeFunction(hi, hj, lij, lik, 1, 1, 1),
    FTildeFunction(hi, hj, lij, lik, 1, 1, 0),
    FFunction(hi, hj, lij, lik, 1, 0, 1),
]

line1 = get_line_Matrix(_expr_mi, vector_system_moments)
line2 = get_line_Matrix(_expr_mj, vector_system_moments)
line3 = get_line_Matrix(_expr_eta_ij, vector_system_moments)
line4 = get_line_Matrix(_expr_eta_ik, vector_system_moments)

In [63]:
M = sp.Matrix([line1, line2, line3, line4])
M[0, :] /= sp.sqrt(lij)*hi*sp.sqrt(hi)/sp.sqrt(hj)
M[1, :] /= sp.sqrt(lij)*sp.sqrt(hi)
M[2, :] /= sp.sqrt(lij)*lij*sp.sqrt(hi)/sp.sqrt(hj)
M[3, :] /= sp.sqrt(lik)/sp.sqrt(hj)

M[0, :] += M[1, :]
M[1, :] += M[2, :]
M[2, :] += M[0, :]/2
M[2, :] -= M[1, :]
M[2, :] += lik*hj*M[3, :]
M[3, :] -= M[2, :]/(lik*hj)
M = M.expand()

M = sp.Matrix(4, 5, lambda i, j: M[i, j].factor())
M[0, :] /= (lij*hi-lik*hj)*hi*sp.sqrt(hj)*2
M[1, :] /= (lij*hi-lik*hj)*lij*sp.sqrt(hj)*2
M[2, :] /= (lij*hi-lik*hj)*sp.sqrt(hj)
M[3, :] *= lik*sp.sqrt(hj)

M[2, :] -= M[0, :]*hi
M[3, :] += M[0, :]*hi**2*lij
M[3, :] += M[1, :]*hj*lij*lik
M[2, :] += M[3, :]/(hi*lij)
M[3, :] += M[2, :]/(lik*hj)*hi**2*lij**2
M = M.expand()

M = sp.Matrix(4, 5, lambda i, j: M[i, j].factor())
M[0, :] /= -2
M[1, :] /= -2
M[2, :] /= -2*lik*hj
M[3, :] /= -2*lij*hi

det_system_moments = sp.Symbol("\\Delta")
det_system_moments_value = M[3, 0]
M[3, 0] = det_system_moments

display(M)

Matrix([
[                             2*\lambda_{ij}*\left<{m_i}\right>*h_i + \lambda_{ij}*\left<{m_j}\right>*h_j + \lambda_{ij} + h_i*h_j, -sqrt(2)*sqrt(\lambda_{ij})*sqrt(h_i)*sqrt(h_j)/2,                                                 0,                           0,                                                                                                                    0],
[                                2*\lambda_{ij}*\left<{\eta_{ij}}\right>*h_i + \lambda_{ij}*h_j + \left<{m_j}\right>*h_i*h_j + h_i,                                                 0, -sqrt(2)*sqrt(\lambda_{ij})*sqrt(h_i)*sqrt(h_j)/2,                           0,                                                                                                                    0],
[2*\lambda_{ij}*\left<{\eta_{ij}}\right> + 2*\lambda_{ik}*\left<{\eta_{ik}}\right> + 2*\lambda_{ik} + 2*\left<{m_i}\right>*h_i + 2,                                                 0,                                             

In [64]:
det_system_moments_value.collect([mi, mj, eta_ij, eta_ik], sp.factor)

\lambda_{ij}**2*h_j + 2*\lambda_{ij}*\lambda_{ik}*\left<{\eta_{ik}}\right>*h_i + 2*\lambda_{ij}*\lambda_{ik}*h_i + 2*\lambda_{ij}*\left<{\eta_{ij}}\right>*(\lambda_{ij}*h_i + \lambda_{ik}*h_j) + 2*\lambda_{ij}*\left<{m_j}\right>*h_i*h_j + 2*\lambda_{ij}*h_i + \lambda_{ik}*h_j + 2*\left<{m_i}\right>*h_i*(\lambda_{ij}*h_i + \lambda_{ik}*h_j) + h_i**2*h_j

In [65]:
result = sp.solve_linear_system(M, *vector_system_moments)

value_system_moments = []
for key in vector_system_moments:
    value_system_moments.append(result[key].factor())

for i in range(4):
    display(sp.Eq(vector_system_moments[i], value_system_moments[i]))
for i in range(4):
    sp.print_latex(sp.Eq(vector_system_moments[i], value_system_moments[i]))

display(sp.Eq(det_system_moments, det_system_moments_value))
sp.print_latex(sp.Eq(det_system_moments, det_system_moments_value))

Eq(cF3(h_i, h_j, \lambda_{ij}, \lambda_{ik}), sqrt(\lambda_{ij})*sqrt(\lambda_{ik})*sqrt(h_i)*sqrt(h_j)*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1)/(2*\Delta))

Eq(FTildeFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1), sqrt(2)*sqrt(\lambda_{ik})*(2*\lambda_{ij}*\left<{m_i}\right>*h_i + \lambda_{ij}*\left<{m_j}\right>*h_j + \lambda_{ij} + h_i*h_j)*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1)/(2*\Delta))

Eq(FTildeFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 0), sqrt(2)*sqrt(\lambda_{ik})*(2*\lambda_{ij}*\left<{\eta_{ij}}\right>*h_i + \lambda_{ij}*h_j + \left<{m_j}\right>*h_i*h_j + h_i)*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1)/(2*\Delta))

Eq(FFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 0, 1), sqrt(2)*sqrt(\lambda_{ij})*sqrt(h_i)*sqrt(h_j)*(\lambda_{ij}*\left<{\eta_{ij}}\right> + \lambda_{ik}*\left<{\eta_{ik}}\right> + \lambda_{ik} + \left<{m_i}\right>*h_i + 1)*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ik}, 1, 1, 1)/(2*\Delta))

{\cal F}^{(3)}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right) = \frac{\sqrt{\lambda_{ij}} \sqrt{\lambda_{ik}} \sqrt{h_{i}} \sqrt{h_{j}} {\cal E}_{123}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right)}{2 \Delta}
\tilde{\cal F}_{123}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right) = \frac{\sqrt{2} \sqrt{\lambda_{ik}} \left(2 \lambda_{ij} \left<{m_i}\right> h_{i} + \lambda_{ij} \left<{m_j}\right> h_{j} + \lambda_{ij} + h_{i} h_{j}\right) {\cal E}_{123}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right)}{2 \Delta}
\tilde{\cal F}_{12}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right) = \frac{\sqrt{2} \sqrt{\lambda_{ik}} \left(2 \lambda_{ij} \left<{\eta_{ij}}\right> h_{i} + \lambda_{ij} h_{j} + \left<{m_j}\right> h_{i} h_{j} + h_{i}\right) {\cal E}_{123}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right)}{2 \Delta}
{\cal F}_{13}\left(h_{i}, h_{j}, \lambda_{ij}, \lambda_{ik}\right) = \frac{\sqrt{2} \sqrt{\lambda_{ij}} \sqrt{h_{i}} \sqrt{h_{j}} \left(\lambda_{ij} \left<{\

Eq(\Delta, 2*\lambda_{ij}**2*\left<{\eta_{ij}}\right>*h_i + \lambda_{ij}**2*h_j + 2*\lambda_{ij}*\lambda_{ik}*\left<{\eta_{ij}}\right>*h_j + 2*\lambda_{ij}*\lambda_{ik}*\left<{\eta_{ik}}\right>*h_i + 2*\lambda_{ij}*\lambda_{ik}*h_i + 2*\lambda_{ij}*\left<{m_i}\right>*h_i**2 + 2*\lambda_{ij}*\left<{m_j}\right>*h_i*h_j + 2*\lambda_{ij}*h_i + 2*\lambda_{ik}*\left<{m_i}\right>*h_i*h_j + \lambda_{ik}*h_j + h_i**2*h_j)

\Delta = 2 \lambda_{ij}^{2} \left<{\eta_{ij}}\right> h_{i} + \lambda_{ij}^{2} h_{j} + 2 \lambda_{ij} \lambda_{ik} \left<{\eta_{ij}}\right> h_{j} + 2 \lambda_{ij} \lambda_{ik} \left<{\eta_{ik}}\right> h_{i} + 2 \lambda_{ij} \lambda_{ik} h_{i} + 2 \lambda_{ij} \left<{m_i}\right> h_{i}^{2} + 2 \lambda_{ij} \left<{m_j}\right> h_{i} h_{j} + 2 \lambda_{ij} h_{i} + 2 \lambda_{ik} \left<{m_i}\right> h_{i} h_{j} + \lambda_{ik} h_{j} + h_{i}^{2} h_{j}


In [66]:
subs_to_moments = dict(zip(vector_system_moments, value_system_moments))
subs_det_system_moments = {det_system_moments: det_system_moments_value}

In [67]:
expr = expr_eta_ik
expr.subs(subs_to_moments).subs(subs_det_system_moments).factor()

\left<{\eta_{ik}}\right>

In [68]:
expr_Q_star = Z3.diff(lij, 2)/(2*Z3) - expr_eta_ik - 1
expr_Q_star = sp.Add(*map(sp.factor, expr_Q_star.expand().args))
# expr_Q_star

In [69]:
expr_diff_hi_lij = Z3.diff(hi, lij)/(Z3)
expr_diff_hi_lij = sp.Add(*map(sp.factor, expr_diff_hi_lij.expand().args))
expr_diff_hi_lij = expr_diff_hi_lij.subs(subs_to_moments)

expr_mj_eta_ij = Z3.diff(hj, lij)/(2*Z3)
expr_mj_eta_ij = sp.Add(*map(sp.factor, expr_mj_eta_ij.expand().args))
expr_mj_eta_ij = expr_mj_eta_ij.subs(subs_to_moments)

expr_mi_eta_ik = Z3.diff(hi, lik)/(2*Z3)
expr_mi_eta_ik = sp.Add(*map(sp.factor, expr_mi_eta_ik.expand().args))
expr_mi_eta_ik = expr_mi_eta_ik.subs(subs_to_moments)

expr_eta_ik_2 = Z3.diff(lik, 2)/(Z3)
expr_eta_ik_2 = sp.Add(*map(sp.factor, expr_eta_ik_2.expand().args))
expr_eta_ik_2 = expr_eta_ik_2.subs(subs_to_moments)

In [70]:
eta_ik_2, mj_eta_ij, mi_eta_ik = Z3sym.eta_ik_2, Z3sym.mj_eta_ij, Z3sym.mi_eta_ik

In [71]:
factor = (lij*hi-lik*hj)**2

expr1 = (expr_diff_hi_lij.factor()*factor).expand() - factor*sp.Dummy()
expr2 = (expr_mj_eta_ij.factor()*factor).expand() - factor*mj_eta_ij
expr3 = (expr_mi_eta_ik.factor()*factor).expand() - factor*mi_eta_ik
expr4 = (expr_eta_ik_2.factor()*factor).expand() - factor*eta_ik_2

In [72]:
vector_system_moments_2 = [
    EFunction(hi, hj, lij, lik, 0, 0, 1)/EFunction(hi, hj, lij, lik, 1, 1, 1),
    EFunction(hi, hj, lij, lik, 1, 0, 0)/EFunction(hi, hj, lij, lik, 1, 1, 1),
    EFunction(hi, hj, lij, lik, 1, 1, 0)/EFunction(hi, hj, lij, lik, 1, 1, 1),
    EFunction(hi, hj, lij, lik, 1, 0, 1)/EFunction(hi, hj, lij, lik, 1, 1, 1),
]

line1 = get_line_Matrix(expr1, vector_system_moments_2)
line2 = get_line_Matrix(expr2, vector_system_moments_2)
line3 = get_line_Matrix(expr3, vector_system_moments_2)
line4 = get_line_Matrix(expr4, vector_system_moments_2)

In [73]:
M = sp.Matrix([line1, line2, line3, line4])
M = M.expand()

M[0, :] += M[1, :]*2/hi*hj
M[1, :] -= M[2, :]*hi**2/hj**2
M[2, :] += M[3, :]/2/hi*lik
M[3, :] += M[2, :]*2*hi/lik

M = M.expand()

display(M)

Matrix([
[                                                      0,                                                                                                                  0,                                                                  \Delta*\lambda_{ij}*h_i/\lambda_{ik} - 2*\Delta*h_j + \Delta*\lambda_{ik}*h_j**2/(\lambda_{ij}*h_i),                                                                                                                  0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [74]:
value_system_moments_2 = []

result = (M[:, :-1]).solve(M[:, -1])
for item in result:
    value_system_moments_2.append(item.factor())

# for i in range(4):
#     display(sp.Eq(vector_system_moments_2[i], value_system_moments_2[i]))
# for i in range(4):
#     sp.print_latex(sp.Eq(vector_system_moments_2[i], value_system_moments_2[i]))

In [75]:
subs_to_moments_2 = dict(zip(vector_system_moments_2, value_system_moments_2))

In [76]:
expr_Q_star_from_m_j_eta_ij = expr_Q_star.subs(subs_to_moments).subs(subs_to_moments_2).subs(subs_det_system_moments).factor().expand()
display(expr_Q_star_from_m_j_eta_ij)
sp.print_latex((lij*expr_Q_star_from_m_j_eta_ij).expand())

-\left<m_j\eta_{ij}\right>*h_j/\lambda_{ij} - 2*\left<{\eta_{ij}}\right>/\lambda_{ij} + \left<{m_i}\right>*h_j/\lambda_{ij}

- \left<m_j\eta_{ij}\right> h_{j} - 2 \left<{\eta_{ij}}\right> + \left<{m_i}\right> h_{j}


In [77]:
subs_to_Q_star = {lij: sp.solve(expr_Q_star_from_m_j_eta_ij-Z3sym.Q_star, lij)[0]}

In [78]:
expr_mjh2 = Z3.diff(hj, 2)/(Z3)
expr_mjh2 = sp.Add(*map(sp.factor, expr_mjh2.expand().args))
expr = expr_mjh2.subs(subs_to_moments).subs(subs_to_moments_2).subs(subs_det_system_moments).factor().expand()

mjh2 = Z3sym.mjh2
display(sp.Eq(mjh2, expr))
sp.print_latex(expr)
 
expr_Q_star_from_mjh2 = expr_Q_star_from_m_j_eta_ij.replace(mj_eta_ij, sp.solve(expr-mjh2, mj_eta_ij)[0]).expand()
display(expr_Q_star_from_mjh2)
sp.print_latex((lij*expr_Q_star_from_mjh2).expand())

_expr_mjh2 = expr

Eq(\left<m_{jh}^2\right>, -2*\lambda_{ij}*\left<m_j\eta_{ij}\right>/h_j + 2*\lambda_{ij}*\left<{m_i}\right>/h_j - 2*\left<{m_j}\right>/h_j + 1)

- \frac{2 \lambda_{ij} \left<m_j\eta_{ij}\right>}{h_{j}} + \frac{2 \lambda_{ij} \left<{m_i}\right>}{h_{j}} - \frac{2 \left<{m_j}\right>}{h_{j}} + 1


-2*\left<{\eta_{ij}}\right>/\lambda_{ij} + \left<m_{jh}^2\right>*h_j**2/(2*\lambda_{ij}**2) + \left<{m_j}\right>*h_j/\lambda_{ij}**2 - h_j**2/(2*\lambda_{ij}**2)

- 2 \left<{\eta_{ij}}\right> + \frac{\left<m_{jh}^2\right> h_{j}^{2}}{2 \lambda_{ij}} + \frac{\left<{m_j}\right> h_{j}}{\lambda_{ij}} - \frac{h_{j}^{2}}{2 \lambda_{ij}}


In [79]:
expr_mj_eta_ik = Z3.diff(hj, lik)/(Z3)
expr_mj_eta_ik = sp.Add(*map(sp.factor, expr_mj_eta_ik.expand().args))
expr = expr_mj_eta_ik.subs(subs_to_moments).subs(subs_to_moments_2).subs(subs_det_system_moments).factor().expand()

expr

_Dummy_1057/2 - \left<m_j\eta_{ij}\right>*h_j/h_i + \left<{m_i}\right>*h_j/h_i - \left<{m_j}\right>

#### Численная проверка

In [80]:
def calc_Z3_symmetrical_integrate(args):
    hi, hj, lij, lik = args
    return Z3num.calc_Z3_integrate(hi, hj, lij, lik)

In [81]:
args_coeffs = hi, hj, lij, lik
args_moments = hi, hj, lij, lik, mi, mj, eta_ij, eta_ik

In [82]:
f_expr_m_i = sp.lambdify(args_coeffs, expr_mi, Z3sym.num_module_norm)
f_expr_m_j = sp.lambdify(args_coeffs, expr_mj, Z3sym.num_module_norm)
f_expr_eta_ij = sp.lambdify(args_coeffs, expr_eta_ij, Z3sym.num_module_norm)
f_expr_eta_ik = sp.lambdify(args_coeffs, expr_eta_ik, Z3sym.num_module_norm)

args = 1.4, 1.1, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(calc_Z3_symmetrical_integrate, args)/(np.asarray([2, 1, 2, 1])*calc_Z3_symmetrical_integrate(args)),
    calc_exprs([f_expr_m_i, f_expr_m_j, f_expr_eta_ij, f_expr_eta_ik], args)
)

0.57547760168819
0.5754693308163132

0.5237828164508537
0.5238038611186142

0.49566723243078437
0.4956543647609227

0.5488518559086242
0.5488711531746446



In [83]:
args = 2, 1, 1.1, 1.4
moments = f_expr_m_i(*args), f_expr_m_j(*args), f_expr_eta_ij(*args), f_expr_eta_ik(*args)

for i in range(4):
    left = vector_system_moments[i].subs(subs_det_system_moments)
    f_left = sp.lambdify(args_moments, left, Z3sym.num_module_norm)
    print(f_left(*args, *moments))

    right = value_system_moments[i].subs(subs_det_system_moments)
    f_right = sp.lambdify(args_moments, right, Z3sym.num_module_norm)
    print(f_right(*args, *moments))

    print()

0.023086804752871965
0.023086804752871934

0.1479250666881099
0.14792506668810984

0.14872104750451542
0.14872104750451573

0.14575565732682025
0.14575565732682022



In [84]:
f_expr_Q_star = sp.lambdify(args_coeffs, expr_Q_star, Z3sym.num_module_norm)

args = 0.2073412, 0.13806503, 1.26262682, 0.15322752
Q_star = nd.directionaldiff(calc_Z3_symmetrical_integrate, args, [0, 0, 1, 0], n=2)/(2*calc_Z3_symmetrical_integrate(args)) - f_expr_eta_ik(*args) - 1
print(Q_star)
print(f_expr_Q_star(*args))

moments = f_expr_m_i(*args), f_expr_m_j(*args), f_expr_eta_ij(*args), f_expr_eta_ik(*args)

f_expr_Q_star_from_m_j_eta_ij = sp.lambdify(list(args_moments) + [mj_eta_ij], expr_Q_star_from_m_j_eta_ij, Z3sym.num_module_norm)
f_expr_m_j_eta_ij = sp.lambdify(args_moments, expr_mj_eta_ij.subs(subs_det_system_moments), Z3sym.num_module_norm)
print(f_expr_Q_star_from_m_j_eta_ij(*args, *moments, f_expr_m_j_eta_ij(*args, *moments)))

f_expr_Q_star_from_m_j_par_2 = sp.lambdify(list(args_moments) + [mjh2], expr_Q_star_from_mjh2, Z3sym.num_module_norm)
f_expr_m_j_par_2 = sp.lambdify(args_moments, expr_mjh2.subs(subs_det_system_moments), Z3sym.num_module_norm)
print(f_expr_Q_star_from_m_j_par_2(*args, *moments, f_expr_m_j_par_2(*args, *moments)))

-0.628904560586544
-0.628902248401817
-0.628902248401817
-0.6289022484018159


### Вырожденный случай

#### Символьные вычисления

In [85]:
_expr = cF3_degenerate(hi, hj, lij, lik, *(4*[0]))

subs_to_cF3Function_degenerate = {
    EFunction(hi, hj, lij, lik, 1, 1, 1): sp.solve(_expr-evaluate_cF3_degenerate(_expr), EFunction(hi, hj, lij, lik, 1, 1, 1))[0]
}

In [86]:
Z3_degenerate_expr = cF3_degenerate(hi, hj, lij, lik, *(4*[0]))/sp.sqrt(hi*hj*lij*lik)

In [87]:
expr_m_i_degenerate = sp.log(Z3_degenerate_expr).diff(hi)/2
expr_m_i_degenerate = expr_m_i_degenerate.expand()
expr_m_i_degenerate = evaluate_cF3_degenerate(expr_m_i_degenerate)
expr_m_i_degenerate = expr_m_i_degenerate.subs(subs_to_cF3Function_degenerate).expand()

expr_m_j_degenerate = sp.log(Z3_degenerate_expr).diff(hj)
expr_m_j_degenerate = expr_m_j_degenerate.expand()
expr_m_j_degenerate = evaluate_cF3_degenerate(expr_m_j_degenerate)
expr_m_j_degenerate = expr_m_j_degenerate.subs(subs_to_cF3Function_degenerate).expand()

expr_eta_ij_degenerate = sp.log(Z3_degenerate_expr).diff(lij)/2
expr_eta_ij_degenerate = expr_eta_ij_degenerate.expand()
expr_eta_ij_degenerate = evaluate_cF3_degenerate(expr_eta_ij_degenerate)
expr_eta_ij_degenerate = expr_eta_ij_degenerate.subs(subs_to_cF3Function_degenerate).expand()

expr_eta_ik_degenerate = sp.log(Z3_degenerate_expr).diff(lik)
expr_eta_ik_degenerate = expr_eta_ik_degenerate.expand()
expr_eta_ik_degenerate = evaluate_cF3_degenerate(expr_eta_ik_degenerate)
expr_eta_ik_degenerate = expr_eta_ik_degenerate.subs(subs_to_cF3Function_degenerate).expand()

In [88]:
n, d = sp.fraction(expr_m_i_degenerate.factor())
_expr_mi = (n-d*mi).expand()

n, d = sp.fraction(expr_m_j_degenerate.factor())
_expr_mj = (n-d*mj).expand()

n, d = sp.fraction(expr_eta_ij_degenerate.factor())
_expr_eta_1 = (n-d*eta_ij).expand()

n, d = sp.fraction(expr_eta_ik_degenerate.factor())
_expr_eta_2 = (n-d*eta_ik).expand()

KeyboardInterrupt: 

In [ ]:
vector_system_moments_degenerate = [
    FFunction(hi, hj, lij, lik, 1, 1, 0),
    FFunction(hi, hj, lij, lik, 1, 0, 1),
    FFunction(hi, hj, lij, lik, 1, 1, 1),
    cF3_degenerate(hi, hj, lij, lik, *(4*[0])),
]

line1 = get_line_Matrix(_expr_mi, vector_system_moments_degenerate)
line2 = get_line_Matrix(_expr_mj, vector_system_moments_degenerate)
line3 = get_line_Matrix(_expr_eta_1, vector_system_moments_degenerate)
line4 = get_line_Matrix(_expr_eta_2, vector_system_moments_degenerate)

In [ ]:
M = sp.Matrix([line1, line2, line3, line4])
M = M.replace(lik, lij*hi/hj)

M[0, :] /= (2*hi**3*hj**2*lij**2)
M[1, :] /= (2*hi**2*hj**3*lij)
M[2, :] /= (2*hi*hj**2*lij**2)
M[3, :] /= (2*hi**2*lij**2)

M[0, :] += M[1, :]/lij
M[1, :] += M[2, :]/hi
M[2, :] += M[3, :]/hj

M = M.expand()

A_system_moments_degenerate = sp.Symbol("A")
A_system_moments_degenerate_value = (M[3, 4]/(lij*hi*hj)).expand()
M[3, 4] = A_system_moments_degenerate*(lij*hi*hj)

display(M)
display(A_system_moments_degenerate_value)
sp.print_latex(A_system_moments_degenerate_value)

Matrix([
[                                                                                                                                                         0,                                                                                                                                                                                                              0,                                                                                                          3*sqrt(2)*sqrt(h_i)*h_j**(3/2)/sqrt(\lambda_{ij}),                                                                                                          -12*\left<{m_i}\right>*h_i*h_j - 6*\left<{m_j}\right>*h_j**2 - 6*h_j - 6*h_i*h_j**2/\lambda_{ij},                      0],
[                                                                                                        3*sqrt(2)*\lambda_{ij}**(3/2)*h_j**(3/2)/sqrt(h_i),                                                                                      

\lambda_{ij}*h_i*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 0, 0, 1) - \lambda_{ij}*h_j*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 1, 0, 0) - h_i*h_j*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 1, 0, 1) + h_j**2*EFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 1, 1, 0)

\lambda_{ij} h_{i} {\cal E}_{3}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right) - \lambda_{ij} h_{j} {\cal E}_{1}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right) - h_{i} h_{j} {\cal E}_{13}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right) + h_{j}^{2} {\cal E}_{12}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right)


In [ ]:
vector_system_moments_degenerate = list(map(lambda item: item.replace(lik, lij*hi/hj), vector_system_moments_degenerate))

result = sp.solve_linear_system(M, *vector_system_moments_degenerate)

value_system_moments_degenerate = []
for key in vector_system_moments_degenerate:
    value_system_moments_degenerate.append(result[key].factor())

n, d = sp.fraction(value_system_moments_degenerate[0])
det_system_moments_degenerate_value = d/2
det_system_moments_degenerate = sp.Symbol("\\Delta")
subs_det_system_moments_degenerate = {det_system_moments_degenerate: det_system_moments_degenerate_value}

value_system_moments_degenerate = list(map(lambda item: item.subs(det_system_moments_degenerate_value, det_system_moments_degenerate), value_system_moments_degenerate))

for i in range(4):
    display(sp.Eq(vector_system_moments_degenerate[i], value_system_moments_degenerate[i]))
for i in range(4):
    sp.print_latex(sp.Eq(vector_system_moments_degenerate[i], value_system_moments_degenerate[i]))

display(sp.Eq(det_system_moments_degenerate, det_system_moments_degenerate_value))
sp.print_latex(sp.Eq(det_system_moments_degenerate, det_system_moments_degenerate_value))

Eq(FFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 1, 1, 0), -sqrt(2)*A*sqrt(\lambda_{ij})*sqrt(h_i)*sqrt(h_j)*(2*\lambda_{ij}*\left<{\eta_{ij}}\right>*h_i + \lambda_{ij}*h_j + \left<{m_j}\right>*h_i*h_j + h_i)/(2*\Delta))

Eq(FFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 1, 0, 1), -sqrt(2)*A*sqrt(\lambda_{ij})*sqrt(h_i)*sqrt(h_j)*(\lambda_{ij}*\left<{\eta_{ij}}\right>*h_j + \lambda_{ij}*\left<{\eta_{ik}}\right>*h_i + \lambda_{ij}*h_i + \left<{m_i}\right>*h_i*h_j + h_j)/(2*\Delta))

Eq(FFunction(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 1, 1, 1), -sqrt(2)*A*sqrt(\lambda_{ij})*sqrt(h_i)*sqrt(h_j)*(2*\lambda_{ij}*\left<{m_i}\right>*h_i + \lambda_{ij}*\left<{m_j}\right>*h_j + \lambda_{ij} + h_i*h_j)/(2*\Delta))

Eq(cF3_degenerate(h_i, h_j, \lambda_{ij}, \lambda_{ij}*h_i/h_j, 0, 0, 0, 0), -A*\lambda_{ij}*h_i*h_j/(2*\Delta))

{\cal F}_{12}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right) = - \frac{\sqrt{2} A \sqrt{\lambda_{ij}} \sqrt{h_{i}} \sqrt{h_{j}} \left(2 \lambda_{ij} \left<{\eta_{ij}}\right> h_{i} + \lambda_{ij} h_{j} + \left<{m_j}\right> h_{i} h_{j} + h_{i}\right)}{2 \Delta}
{\cal F}_{13}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right) = - \frac{\sqrt{2} A \sqrt{\lambda_{ij}} \sqrt{h_{i}} \sqrt{h_{j}} \left(\lambda_{ij} \left<{\eta_{ij}}\right> h_{j} + \lambda_{ij} \left<{\eta_{ik}}\right> h_{i} + \lambda_{ij} h_{i} + \left<{m_i}\right> h_{i} h_{j} + h_{j}\right)}{2 \Delta}
{\cal F}_{123}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\right) = - \frac{\sqrt{2} A \sqrt{\lambda_{ij}} \sqrt{h_{i}} \sqrt{h_{j}} \left(2 \lambda_{ij} \left<{m_i}\right> h_{i} + \lambda_{ij} \left<{m_j}\right> h_{j} + \lambda_{ij} + h_{i} h_{j}\right)}{2 \Delta}
{\cal F}_{deg,0, 0, 0, 0}\left(h_{i}, h_{j}, \lambda_{ij}, \frac{\lambda_{ij} h_{i}}{h_{j}}\righ

Eq(\Delta, \lambda_{ij}**3*\left<{\eta_{ij}}\right>*h_j**3 - 2*\lambda_{ij}**3*\left<{\eta_{ik}}\right>*h_i**3 + \lambda_{ij}**3*\left<{\eta_{ik}}\right>*h_i*h_j**2 - 2*\lambda_{ij}**3*h_i**3 + 2*\lambda_{ij}**3*h_i*h_j**2 - \lambda_{ij}**2*\left<{\eta_{ij}}\right>*h_i*h_j**2 + \lambda_{ij}**2*\left<{\eta_{ik}}\right>*h_i**2*h_j - \lambda_{ij}**2*\left<{m_i}\right>*h_i*h_j**3 + 2*\lambda_{ij}**2*\left<{m_j}\right>*h_i**2*h_j**2 - \lambda_{ij}**2*\left<{m_j}\right>*h_j**4 - \lambda_{ij}*\left<{\eta_{ij}}\right>*h_i**2*h_j**3 + \lambda_{ij}*\left<{\eta_{ik}}\right>*h_i**3*h_j**2 - \lambda_{ij}*\left<{m_i}\right>*h_i**2*h_j**2 + \lambda_{ij}*\left<{m_j}\right>*h_i*h_j**3 + 2*\lambda_{ij}*h_i**3*h_j**2 - 2*\lambda_{ij}*h_i*h_j**4 + \left<{m_i}\right>*h_i**3*h_j**3 - \left<{m_j}\right>*h_i**2*h_j**4)

\Delta = \lambda_{ij}^{3} \left<{\eta_{ij}}\right> h_{j}^{3} - 2 \lambda_{ij}^{3} \left<{\eta_{ik}}\right> h_{i}^{3} + \lambda_{ij}^{3} \left<{\eta_{ik}}\right> h_{i} h_{j}^{2} - 2 \lambda_{ij}^{3} h_{i}^{3} + 2 \lambda_{ij}^{3} h_{i} h_{j}^{2} - \lambda_{ij}^{2} \left<{\eta_{ij}}\right> h_{i} h_{j}^{2} + \lambda_{ij}^{2} \left<{\eta_{ik}}\right> h_{i}^{2} h_{j} - \lambda_{ij}^{2} \left<{m_i}\right> h_{i} h_{j}^{3} + 2 \lambda_{ij}^{2} \left<{m_j}\right> h_{i}^{2} h_{j}^{2} - \lambda_{ij}^{2} \left<{m_j}\right> h_{j}^{4} - \lambda_{ij} \left<{\eta_{ij}}\right> h_{i}^{2} h_{j}^{3} + \lambda_{ij} \left<{\eta_{ik}}\right> h_{i}^{3} h_{j}^{2} - \lambda_{ij} \left<{m_i}\right> h_{i}^{2} h_{j}^{2} + \lambda_{ij} \left<{m_j}\right> h_{i} h_{j}^{3} + 2 \lambda_{ij} h_{i}^{3} h_{j}^{2} - 2 \lambda_{ij} h_{i} h_{j}^{4} + \left<{m_i}\right> h_{i}^{3} h_{j}^{3} - \left<{m_j}\right> h_{i}^{2} h_{j}^{4}


In [ ]:
expr = det_system_moments_degenerate_value.collect([mi, mj, eta_ij, eta_ik], sp.factor)
display(expr)
sp.print_latex(expr)
det_system_moments_degenerate_value.subs({hj: hi, mj: mi, eta_ik: eta_ij})

\lambda_{ij}*\left<{\eta_{ij}}\right>*h_j**2*(\lambda_{ij}**2*h_j - \lambda_{ij}*h_i - h_i**2*h_j) - \lambda_{ij}*\left<{\eta_{ik}}\right>*h_i*(2*\lambda_{ij}**2*h_i**2 - \lambda_{ij}**2*h_j**2 - \lambda_{ij}*h_i*h_j - h_i**2*h_j**2) - 2*\lambda_{ij}*h_i*(\lambda_{ij} - h_j)*(\lambda_{ij} + h_j)*(h_i - h_j)*(h_i + h_j) - \left<{m_i}\right>*h_i*h_j**2*(\lambda_{ij}**2*h_j + \lambda_{ij}*h_i - h_i**2*h_j) + \left<{m_j}\right>*h_j**2*(2*\lambda_{ij}**2*h_i**2 - \lambda_{ij}**2*h_j**2 + \lambda_{ij}*h_i*h_j - h_i**2*h_j**2)

\lambda_{ij} \left<{\eta_{ij}}\right> h_{j}^{2} \left(\lambda_{ij}^{2} h_{j} - \lambda_{ij} h_{i} - h_{i}^{2} h_{j}\right) - \lambda_{ij} \left<{\eta_{ik}}\right> h_{i} \left(2 \lambda_{ij}^{2} h_{i}^{2} - \lambda_{ij}^{2} h_{j}^{2} - \lambda_{ij} h_{i} h_{j} - h_{i}^{2} h_{j}^{2}\right) - 2 \lambda_{ij} h_{i} \left(\lambda_{ij} - h_{j}\right) \left(\lambda_{ij} + h_{j}\right) \left(h_{i} - h_{j}\right) \left(h_{i} + h_{j}\right) - \left<{m_i}\right> h_{i} h_{j}^{2} \left(\lambda_{ij}^{2} h_{j} + \lambda_{ij} h_{i} - h_{i}^{2} h_{j}\right) + \left<{m_j}\right> h_{j}^{2} \left(2 \lambda_{ij}^{2} h_{i}^{2} - \lambda_{ij}^{2} h_{j}^{2} + \lambda_{ij} h_{i} h_{j} - h_{i}^{2} h_{j}^{2}\right)


0

In [ ]:
subs_to_A_degenerate = {EFunction(hi, hj, lij, lij*hi/hj, 0, 0, 1): sp.solve(A_system_moments_degenerate_value-A_system_moments_degenerate, EFunction(hi, hj, lij, lij*hi/hj, 0, 0, 1))[0]}
subs_to_moments_degenerate = dict(zip(vector_system_moments_degenerate, value_system_moments_degenerate))

In [ ]:
expr = expr_eta_ik_degenerate.replace(lik, lij*hi/hj)
expr.subs(subs_to_A_degenerate).subs(subs_to_moments_degenerate).subs(subs_det_system_moments_degenerate).factor()

\left<{\eta_{ik}}\right>

In [ ]:
def calc_degenerate(expr):
    expr = expr.expand()
    expr = evaluate_cF3_degenerate(expr)
    expr = expr.subs(subs_to_cF3Function_degenerate).expand()
    expr = expr.replace(lik, lij*hi/hj)
    expr = expr.subs(subs_to_A_degenerate).subs(subs_to_moments_degenerate).expand()
    return expr

In [ ]:
expr_Q_star_degenerate = calc_degenerate(Z3_degenerate_expr.diff(lij, 2)/(2*Z3_degenerate_expr) - eta_ik - 1)

# expr_Q_star_degenerate

In [ ]:
expr_diff_h_i_l_ij_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hi, lij)/(Z3_degenerate_expr))
expr_m_j_eta_ij_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hj, lij)/(2*Z3_degenerate_expr))
expr_m_i_eta_ik_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hi, lik)/(2*Z3_degenerate_expr))
expr_eta_ik_2_degenerate = calc_degenerate(Z3_degenerate_expr.diff(lik, 2)/(Z3_degenerate_expr))

In [ ]:
vector_system_moments_2_degenerate = [
    EFunction(hi, hj, lij, lij*hi/hj, 0, 0, 0)/A_system_moments_degenerate,
    EFunction(hi, hj, lij, lij*hi/hj, 1, 1, 0)/A_system_moments_degenerate,
    EFunction(hi, hj, lij, lij*hi/hj, 1, 0, 1)/A_system_moments_degenerate,
    EFunction(hi, hj, lij, lij*hi/hj, 1, 0, 0)/A_system_moments_degenerate
]

line1 = get_line_Matrix(expr_diff_h_i_l_ij_degenerate-sp.Dummy(), vector_system_moments_2_degenerate)
line2 = get_line_Matrix(expr_m_j_eta_ij_degenerate-mj_eta_ij, vector_system_moments_2_degenerate)
line3 = get_line_Matrix(expr_m_i_eta_ik_degenerate-mi_eta_ik, vector_system_moments_2_degenerate)
line4 = get_line_Matrix(expr_eta_ik_2_degenerate-eta_ik_2, vector_system_moments_2_degenerate)

In [ ]:
M = sp.Matrix([line1, line2, line3, line4])

M[0, :] += M[1, :]*2/hi*hj
M[1, :] -= M[2, :]/hj**2*hi**2
M[2, :] += M[3, :]*lij/hj/2

M[1, :] -= M[0, :]/hj*hi/2
M[2, :] += M[1, :]*hj**2/2/hi**2

M = M.expand()

display(M)

Matrix([
[                                    0,                                                                                                                                          -\Delta/(\lambda_{ij}**2*h_i**2),                                                                                    0,                                                                                   0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
value_system_moments_2_degenerate = []

value_system_moments_2_degenerate.append((M[0, -1]/M[0, 1]).factor())
value_system_moments_2_degenerate.append((M[1, -1]/M[1, 2]).factor())
value_system_moments_2_degenerate.append((M[2, -1]/M[2, 3]).factor())

expr = sum([-M[-1, i+1]*value_system_moments_2_degenerate[i] for i in range(3)])+M[-1, -1]
value_system_moments_2_degenerate.insert(0, (expr/M[-1, 0]).factor())

# for i in range(4):
#     display(sp.Eq(vector_system_moments_2_degenerate[i], value_system_moments_2_degenerate[i]))
# for i in range(4):
#     sp.print_latex(sp.Eq(vector_system_moments_2_degenerate[i], value_system_moments_2_degenerate[i]))

In [ ]:
subs_to_moments_2_degenerate = dict(zip(vector_system_moments_2_degenerate, value_system_moments_2_degenerate))

In [ ]:
expr_eta_ik_2_degenerate.subs(subs_to_moments_2_degenerate).factor()

\left<\eta_{ik}^{2}\right>

In [ ]:
expr_Q_star_degenerate_from_m_j_eta_ij = expr_Q_star_degenerate.subs(subs_to_moments_2_degenerate).subs(subs_det_system_moments_degenerate).factor().expand()
expr_Q_star_degenerate_from_m_j_eta_ij

-\left<m_j\eta_{ij}\right>*h_j/\lambda_{ij} - 2*\left<{\eta_{ij}}\right>/\lambda_{ij} + \left<{m_i}\right>*h_j/\lambda_{ij}

In [ ]:
expr_m_j_par_2_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hj, 2)/(Z3_degenerate_expr))
expr = expr_m_j_par_2_degenerate.subs(subs_to_moments_2_degenerate).subs(subs_det_system_moments_degenerate).factor().expand()

display(sp.Eq(mjh2, expr))

Eq(\left<m_{jh}^2\right>, -2*\lambda_{ij}*\left<m_j\eta_{ij}\right>/h_j + 2*\lambda_{ij}*\left<{m_i}\right>/h_j - 2*\left<{m_j}\right>/h_j + 1)

In [ ]:
expr_m_j_eta_ik_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hj, lik)/(Z3_degenerate_expr))
expr = expr_m_j_eta_ik_degenerate.subs(subs_to_moments_2_degenerate).subs(subs_det_system_moments_degenerate).factor().expand()
expr

_Dummy_1813/2 - \left<m_j\eta_{ij}\right>*h_j/h_i + \left<{m_i}\right>*h_j/h_i - \left<{m_j}\right>

#### Проверка

In [ ]:
to_cF3 = lambda *args: cF3(*args[:4])

f_expr_m_i_degenerate = sp.lambdify(args_coeffs, expr_m_i_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)
f_expr_m_j_degenerate = sp.lambdify(args_coeffs, expr_m_j_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)
f_expr_eta_ij_degenerate = sp.lambdify(args_coeffs, expr_eta_ij_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)
f_expr_eta_ik_degenerate = sp.lambdify(args_coeffs, expr_eta_ik_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)

args = 1, 2, 2., 1
compare_exprs(
    calc_numerical_diffs(calc_Z3_symmetrical_integrate, args)/(np.asarray([2, 1, 2, 1])*calc_Z3_symmetrical_integrate(args)),
    calc_exprs([f_expr_m_i_degenerate, f_expr_m_j_degenerate, f_expr_eta_ij_degenerate, f_expr_eta_ik_degenerate], args)
)

0.5919397633608484
0.5919388922079462

0.6787309180732718
0.6787291810795726

0.6787309180733092
0.6787291810795717

0.5919397633608192
0.5919388922079476



In [ ]:
f_expr_m_i_degenerate(*args)

0.5919388922079462

In [ ]:
args_moments = hi, hj, lij, lik, mi, mj, eta_ij, eta_ik

In [ ]:
args = 2, 1, 1.1
args = list(args) + [args[2]*args[0]/args[1]]
moments = f_expr_m_i_degenerate(*args), f_expr_m_j_degenerate(*args), f_expr_eta_ij_degenerate(*args), f_expr_eta_ik_degenerate(*args)

for i in range(4):
    left = vector_system_moments_degenerate[i].replace(cF3_degenerate, to_cF3).subs(subs_det_system_moments_degenerate)
    f_left = sp.lambdify(args_moments, left, Z3sym.num_module_norm)
    print(f_left(*args, *moments))

    right = value_system_moments_degenerate[i].replace(A_system_moments_degenerate, A_system_moments_degenerate_value).subs(subs_det_system_moments_degenerate)
    # right = A_system_moments_degenerate_value
    f_right = sp.lambdify(args_moments, right, Z3sym.num_module_norm)
    print(f_right(*args, *moments))

    print()

0.14645871513843645
0.14645871513844133

0.14575565732682025
0.14575565732682452

0.14576718452911774
0.14576718452912232

0.02239976480464764
0.02239976480464833



### Вырожденный случай с $h_1 = h_2$ и $\lambda_1 = \lambda_2$

#### Символьные вычисления

In [ ]:
class EFunction_spec_1(EFunction):
    @classmethod
    def eval(cls, *args):
        arg = args[4] + args[5] + args[6]

        if arg == 1 and not args[4]:
            return EFunction_spec_1(*args[:4], 1, 0, 0)
        
        if arg == 2 and (not args[4] or not args[5]):
            return EFunction_spec_1(*args[:4], 1, 1, 0)


class FFunction_spec_1(FFunction):
    @classmethod
    def eval(cls, *args):
        arg = args[4] + args[5] + args[6]

        if arg == 1 and not args[4]:
            return FFunction_spec_1(*args[:4], 1, 0, 0)
        
        if arg == 2 and (not args[4] or not args[5]):
            return FFunction_spec_1(*args[:4], 1, 1, 0)

In [ ]:
h, l = sp.symbols("h \\lambda")
m = sp.Symbol("\\left<m\\right>")
eta = sp.Symbol("\\left<\eta\\right>")

In [ ]:
_expr = cF3_degenerate(h, h, l, l, *(4*[0]))

subs_to_cF3Function_degenerate_spec_1 = {
    EFunction_spec_1(h, h, l, l, 1, 1, 1): sp.solve(_expr-evaluate_cF3_degenerate(_expr).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1), EFunction_spec_1(h, h, l, l, 1, 1, 1))[0]
}

In [ ]:
subs_to_cF3Function_degenerate_spec_1

{EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1): 3*sqrt(2)*sqrt(\lambda)*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0) - 2*cF3_degenerate(h, h, \lambda, \lambda, 0, 0, 0, 0) + sqrt(2)*h*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/sqrt(\lambda)}

In [ ]:
expr_m_i_degenerate_spec_1 = evaluate_cF3_degenerate(expr_m_i_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_m_i_degenerate_spec_1 = expr_m_i_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

expr_m_j_degenerate_spec_1 = evaluate_cF3_degenerate(expr_m_j_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_m_j_degenerate_spec_1 = expr_m_j_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

expr_eta_ij_degenerate_spec_1 = evaluate_cF3_degenerate(expr_eta_ij_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_eta_ij_degenerate_spec_1 = expr_eta_ij_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

expr_eta_ik_degenerate_spec_1 = evaluate_cF3_degenerate(expr_eta_ik_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_eta_ik_degenerate_spec_1 = expr_eta_ik_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

In [ ]:
expr_m_i_degenerate_spec_1.subs(subs_to_cF3Function_degenerate)

-1/(3*h) - h/(3*\lambda) + sqrt(2)*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(6*sqrt(\lambda)*(3*sqrt(2)*sqrt(\lambda)*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0)/2 - EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/2 + sqrt(2)*h*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(2*sqrt(\lambda))))

In [ ]:
expr_eta_ik_degenerate_spec_1

-1/2 - 1/(3*\lambda) + h**2/(6*\lambda**2) + sqrt(2)*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0)/(4*sqrt(\lambda)*(3*sqrt(2)*sqrt(\lambda)*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0)/2 - EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/2 + sqrt(2)*h*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(2*sqrt(\lambda)))) - sqrt(2)*h*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(12*\lambda**(3/2)*(3*sqrt(2)*sqrt(\lambda)*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0)/2 - EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/2 + sqrt(2)*h*FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(2*sqrt(\lambda))))

In [ ]:
n, d = sp.fraction(expr_m_i_degenerate_spec_1.factor())
_expr_m = (n-d*m).expand()

n, d = sp.fraction(expr_eta_ij_degenerate_spec_1.factor())
_expr_eta_ij = (n-d*eta).expand()

In [ ]:
vector_system_moments_degenerate_spec_1 = [
    FFunction_spec_1(h, h, l, l, 1, 1, 1),
    FFunction_spec_1(h, h, l, l, 1, 1, 0),
]

line1 = get_line_Matrix(_expr_m, vector_system_moments_degenerate_spec_1)
line2 = get_line_Matrix(_expr_eta_ij, vector_system_moments_degenerate_spec_1)

In [ ]:
result = sp.solve_linear_system(sp.Matrix([line1, line2]), *vector_system_moments_degenerate_spec_1)

value_system_moments_degenerate_spec_1 = []
for key in vector_system_moments_degenerate_spec_1:
    value_system_moments_degenerate_spec_1.append(result[key].factor())

for i in range(2):
    display(sp.Eq(vector_system_moments_degenerate_spec_1[i], value_system_moments_degenerate_spec_1[i]))
for i in range(2):
    sp.print_latex(sp.Eq(vector_system_moments_degenerate_spec_1[i], value_system_moments_degenerate_spec_1[i]))

Eq(FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1), sqrt(2)*sqrt(\lambda)*(3*\lambda*\left<m\right>*h + \lambda + h**2)*EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(2*h*(6*\lambda**2*\left<\eta\right> + 3*\lambda**2 + 6*\lambda*\left<m\right>*h + 3*\lambda + h**2)))

Eq(FFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0), sqrt(2)*sqrt(\lambda)*(2*\lambda*\left<\eta\right> + \lambda + \left<m\right>*h + 1)*EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1)/(2*(6*\lambda**2*\left<\eta\right> + 3*\lambda**2 + 6*\lambda*\left<m\right>*h + 3*\lambda + h**2)))

{\cal F}_{123}\left(h, h, \lambda, \lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} \left(3 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{123}\left(h, h, \lambda, \lambda\right)}{2 h \left(6 \lambda^{2} \left<\eta\right> + 3 \lambda^{2} + 6 \lambda \left<m\right> h + 3 \lambda + h^{2}\right)}
{\cal F}_{12}\left(h, h, \lambda, \lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} \left(2 \lambda \left<\eta\right> + \lambda + \left<m\right> h + 1\right) {\cal E}_{123}\left(h, h, \lambda, \lambda\right)}{2 \left(6 \lambda^{2} \left<\eta\right> + 3 \lambda^{2} + 6 \lambda \left<m\right> h + 3 \lambda + h^{2}\right)}


In [ ]:
subs_to_moments_degenerate_spec_1 = dict(zip(vector_system_moments_degenerate_spec_1, value_system_moments_degenerate_spec_1))

In [ ]:
expr = expr_eta_ik_degenerate_spec_1
expr.subs(subs_to_moments_degenerate_spec_1).factor()

\left<\eta\right>

In [ ]:
def calc_degenerate_spec_1(expr):
    expr = expr.expand()
    expr = evaluate_cF3_degenerate(expr)
    expr = expr.replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
    expr = expr.subs({hi: h, hj: h, lij: l, lik: l})
    expr = expr.subs(subs_to_moments_degenerate_spec_1).factor().expand()
    return expr

In [ ]:
expr_Q_star_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(lij, 2)/(2*Z3_degenerate_expr) - eta - 1)
# expr_Q_star_degenerate_spec_1

In [ ]:
expr_eta_ik_2_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(lik, 2)/Z3_degenerate_expr)
expr_m_j_eta_ik_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, lik)/Z3_degenerate_expr)
expr_m_j_eta_ij_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, lij)/(2*Z3_degenerate_expr))
expr_m_j_par_2_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, hj)/Z3_degenerate_expr)

In [ ]:
vector_system_moments_2_degenerate_spec_1 = [
    EFunction_spec_1(h, h, l, l, 1, 1, 0)/EFunction_spec_1(h, h, l, l, 1, 1, 1),
    EFunction_spec_1(h, h, l, l, 1, 0, 0)/EFunction_spec_1(h, h, l, l, 1, 1, 1),
    EFunction_spec_1(h, h, l, l, 0, 0, 0)/EFunction_spec_1(h, h, l, l, 1, 1, 1),
]

mj_eta_ik = sp.Symbol("\\left<m_j\\eta_{ik}\\right>")
line1 = get_line_Matrix(expr_eta_ik_2_degenerate_spec_1-eta_ik_2, vector_system_moments_2_degenerate_spec_1)
line2 = get_line_Matrix(expr_m_j_eta_ik_degenerate_spec_1-mj_eta_ik, vector_system_moments_2_degenerate_spec_1)
line3 = get_line_Matrix(expr_m_j_par_2_degenerate_spec_1-mjh2, vector_system_moments_2_degenerate_spec_1)

In [ ]:
M = sp.Matrix([line1, line2, line3])

M[0, :] *= 10*l**4
M[1, :] *= 10*l**3
M[2, :] *= 10*l**2*h**2

M[1, :] -= M[2, :]/h
M[0, :] -= M[2, :]
M[0, :] -= M[1, :]*h
M[2, :] -= M[0, :]/5*2
M[2, :] -= M[0, :]/5*2*h**2/l
M[2, :] -= M[1, :]/5*3*h*l
M[2, :] += M[1, :]/5*3*h
M[2, :] += M[1, :]/5/l*h**3

M = M.expand()

display(M)

Matrix([
[                                                                                                                0,                                                                                                                  -30*\lambda**4*\left<\eta\right> - 15*\lambda**4 - 30*\lambda**3*\left<m\right>*h - 15*\lambda**3 - 5*\lambda**2*h**2,                                                                                                                            0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          -35*\lambda*

In [ ]:
value_system_moments_2_degenerate_spec_1 = []

result = (M[:, :-1]).solve(M[:, -1])
for item in result:
    value_system_moments_2_degenerate_spec_1.append(item.factor())

for i in range(3):
    display(sp.Eq(vector_system_moments_2_degenerate_spec_1[i], value_system_moments_2_degenerate_spec_1[i]))
for i in range(3):
    sp.print_latex(sp.Eq(vector_system_moments_2_degenerate_spec_1[i], value_system_moments_2_degenerate_spec_1[i]))

Eq(EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 0)/EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1), (2*\lambda**2*\left<\eta\right> + 7*\lambda**2*\left<m\right>*h + 2*\lambda**2*\left<m_j\eta_{ik}\right>*h + \lambda**2 + 4*\lambda*\left<\eta\right>*h**2 - \lambda*\left<m\right>*h - 2*\lambda*\left<m_{jh}^2\right>*h**2 + 4*\lambda*h**2 + \lambda + \left<m\right>*h**3 + h**2)/(h*(6*\lambda**2*\left<\eta\right> + 3*\lambda**2 + 6*\lambda*\left<m\right>*h + 3*\lambda + h**2)))

Eq(EFunction_spec_1(h, h, \lambda, \lambda, 1, 0, 0)/EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1), (7*\lambda**2*\left<\eta\right> - 2*\lambda**2*\left<\eta_{ik}^{2}\right> + 4*\lambda**2 - \lambda*\left<\eta\right> + 4*\lambda*\left<m\right>*h + 2*\lambda*\left<m_j\eta_{ik}\right>*h + 2*\lambda + \left<\eta\right>*h**2 + 2*\left<m\right>*h + 1)/(6*\lambda**2*\left<\eta\right> + 3*\lambda**2 + 6*\lambda*\left<m\right>*h + 3*\lambda + h**2))

Eq(EFunction_spec_1(h, h, \lambda, \lambda, 0, 0, 0)/EFunction_spec_1(h, h, \lambda, \lambda, 1, 1, 1), (3*\lambda**3*\left<\eta\right> - 2*\lambda**3*\left<\eta_{ik}^{2}\right> + 6*\lambda**3*\left<m\right>*h + 3*\lambda**3*\left<m_j\eta_{ik}\right>*h + 2*\lambda**3 + 6*\lambda**2*\left<\eta\right>*h**2 - \lambda**2*\left<\eta\right> - 2*\lambda**2*\left<\eta_{ik}^{2}\right>*h**2 - \lambda**2*\left<m_j\eta_{ik}\right>*h - 3*\lambda**2*\left<m_{jh}^2\right>*h**2 + 5*\lambda**2*h**2 - \lambda*\left<\eta\right>*h**2 - \lambda*\left<m\right>*h + \lambda*\left<m_j\eta_{ik}\right>*h**3 - 2*\lambda*\left<m_{jh}^2\right>*h**2 + 2*\lambda*h**2 + \lambda + 2*\left<m\right>*h**3 + \left<m_{jh}^2\right>*h**4 - h**4)/(\lambda*h*(6*\lambda**2*\left<\eta\right> + 3*\lambda**2 + 6*\lambda*\left<m\right>*h + 3*\lambda + h**2)))

\frac{{\cal E}_{12}\left(h, h, \lambda, \lambda\right)}{{\cal E}_{123}\left(h, h, \lambda, \lambda\right)} = \frac{2 \lambda^{2} \left<\eta\right> + 7 \lambda^{2} \left<m\right> h + 2 \lambda^{2} \left<m_j\eta_{ik}\right> h + \lambda^{2} + 4 \lambda \left<\eta\right> h^{2} - \lambda \left<m\right> h - 2 \lambda \left<m_{jh}^2\right> h^{2} + 4 \lambda h^{2} + \lambda + \left<m\right> h^{3} + h^{2}}{h \left(6 \lambda^{2} \left<\eta\right> + 3 \lambda^{2} + 6 \lambda \left<m\right> h + 3 \lambda + h^{2}\right)}
\frac{{\cal E}_{1}\left(h, h, \lambda, \lambda\right)}{{\cal E}_{123}\left(h, h, \lambda, \lambda\right)} = \frac{7 \lambda^{2} \left<\eta\right> - 2 \lambda^{2} \left<\eta_{ik}^{2}\right> + 4 \lambda^{2} - \lambda \left<\eta\right> + 4 \lambda \left<m\right> h + 2 \lambda \left<m_j\eta_{ik}\right> h + 2 \lambda + \left<\eta\right> h^{2} + 2 \left<m\right> h + 1}{6 \lambda^{2} \left<\eta\right> + 3 \lambda^{2} + 6 \lambda \left<m\right> h + 3 \lambda + h^{2}}
\frac{{\cal E}_{}\left

In [ ]:
subs_to_moments_2_degenerate_spec_1 = dict(zip(vector_system_moments_2_degenerate_spec_1, value_system_moments_2_degenerate_spec_1))

In [ ]:
expr_m_j_eta_ik_degenerate_spec_1.subs(subs_to_moments_2_degenerate_spec_1).factor()

\left<m_j\eta_{ik}\right>

In [ ]:
expr_Q_star_degenerate_spec_1_from_moments_2 = expr_Q_star_degenerate_spec_1.subs(subs_to_moments_2_degenerate_spec_1).factor().expand()
expr_Q_star_degenerate_spec_1_from_moments_2

-2*\left<\eta\right>/\lambda + \left<m\right>*h/\lambda**2 + \left<m_{jh}^2\right>*h**2/(2*\lambda**2) - h**2/(2*\lambda**2)

In [ ]:
expr = calc_degenerate_spec_1(expr_m_j_eta_ij_degenerate_spec_1)
expr.subs(subs_to_moments_2_degenerate_spec_1).factor().expand()

\left<m\right> - \left<m\right>/\lambda - \left<m_{jh}^2\right>*h/(2*\lambda) + h/(2*\lambda)

In [ ]:
expr = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hi, lik)/(2*Z3_degenerate_expr))
expr.subs(subs_to_moments_2_degenerate_spec_1).factor().expand()

\left<m\right> - \left<m\right>/\lambda - \left<m_{jh}^2\right>*h/(2*\lambda) + h/(2*\lambda)

#### Проверка

In [ ]:
to_cF3 = lambda *args: cF3(*args[:4])
subs = {FFunction_spec_1: FFunction, EFunction_spec_1: EFunction}

f_expr_m_i_degenerate_spec_1 = sp.lambdify((h, l), expr_m_i_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)
f_expr_m_j_degenerate_spec_1 = sp.lambdify((h, l), expr_m_j_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)
f_expr_eta_ij_degenerate_spec_1 = sp.lambdify((h, l), expr_eta_ij_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)
f_expr_eta_ik_degenerate_spec_1 = sp.lambdify((h, l), expr_eta_ik_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)

args = 1, 2.
compare_exprs(
    calc_numerical_diffs(calc_Z3_symmetrical_integrate, [args[0], args[0], args[1], args[1]])/(np.asarray([2, 1, 2, 1])*calc_Z3_symmetrical_integrate([args[0], args[0], args[1], args[1]])),
    calc_exprs([f_expr_m_i_degenerate_spec_1, f_expr_m_j_degenerate_spec_1, f_expr_eta_ij_degenerate_spec_1, f_expr_eta_ik_degenerate_spec_1], args)
)

0.5642439350057784
0.5642443571711042

0.5642439350057372
0.5642443571711042

0.6940680018488145
0.6940685270916233

0.6940680018488555
0.6940685270916233



In [ ]:
args = 1, 2.
moments = f_expr_m_i_degenerate_spec_1(*args), f_expr_eta_ij_degenerate_spec_1(*args),

for i in range(2):
    left = vector_system_moments_degenerate_spec_1[i].subs(subs)
    f_left = sp.lambdify((h, l, m, eta), left, Z3sym.num_module_norm)
    print(f_left(*args, *moments))

    right = vector_system_moments_degenerate_spec_1[i].subs(subs)
    f_right = sp.lambdify((h, l, m, eta), right, Z3sym.num_module_norm)
    print(f_right(*args, *moments))

    print()

0.15010840845148632
0.15010840845148632

0.1490517864006691
0.1490517864006691



In [ ]:
f_expr_Q_star_degenerate_spec_1 = sp.lambdify((h, l, m, eta), expr_Q_star_degenerate_spec_1.factor().subs(subs), Z3sym.num_module_norm)
f_expr_Q_star_degenerate_spec_1_from_moments_2 = sp.lambdify((h, l, m, eta, mjh2), expr_Q_star_degenerate_spec_1_from_moments_2.subs(subs), Z3sym.num_module_norm)
f_expr_m_par_2_degenerate_spec_1 = sp.lambdify((h, l, m, eta), expr_m_j_par_2_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)

args = 49.03486660449675, 0.510352248405618
moments = f_expr_m_i_degenerate_spec_1(*args), f_expr_eta_ij_degenerate_spec_1(*args),

Q_star = nd.directionaldiff(calc_Z3_symmetrical_integrate, [args[0], args[0], args[1], args[1]], [0, 0, 1, 0], n=2)
Q_star /= 2*calc_Z3_symmetrical_integrate([args[0], args[0], args[1], args[1]])
Q_star -= f_expr_eta_ik_degenerate_spec_1(*args) + 1

print(Q_star)
print(f_expr_Q_star_degenerate_spec_1(*args, *moments))
print(f_expr_Q_star_degenerate_spec_1_from_moments_2(*args, *moments, f_expr_m_par_2_degenerate_spec_1(*args, *moments)))

-3.031110959723879
10846.749138665664
10846.749138755826


## Генерация кода для метода Ньютона

### Определения

In [92]:
class CPrinter(symbase.CPrinter):
    def _print_Function(self, expr):
        if isinstance(expr, Z2sym.F2):
            return f"dawson({self._print(expr.args[0])})"
        
        if isinstance(expr, F3):
            return f"F3({self._print(expr.args[0])}, {self._print(expr.args[1])}, {self._print(expr.args[2])})"

        return super()._print_Function(expr)
    

def get_ccode(expr):
    expr = symbase.optimize_pow(expr, {hi: "h_i", hj: "h_j", lij: "l_ij", lik: "l_ik"})
    return CPrinter().doprint(expr)

### Общий случай

In [93]:
subs_exp = {
    sp.exp(-2*hi): sp.Symbol("_exp_2h_i"),
    sp.exp(-2*hj): sp.Symbol("_exp_2h_j"),
    sp.exp(-2*lij): sp.Symbol("_exp_2l_ij"),
    sp.exp(-2*lik): sp.Symbol("_exp_2l_ik")
}

def to_norm(expr):
    expr *= sp.exp(-hj-lik-2*(hi+lij))
    return expr.expand()

In [95]:
print("kappa = l_ij*h_i - l_ik*h_j")
print("_kappa = 1/kappa")
print("_h_i, _h_j, _l_ij, _l_ik = 1/h_i, 1/h_j, 1/l_ij, 1/l_ik")
print("sqrt_h_i, sqrt_h_j, sqrt_l_ij, sqrt_l_ik = np.sqrt(h_i), np.sqrt(h_j), np.sqrt(l_ij), np.sqrt(l_ik)")
print("_sqrt_h_i, _sqrt_h_j, _sqrt_l_ij, _sqrt_l_ik = 1/sqrt_h_i, 1/sqrt_h_j, 1/sqrt_l_ij, 1/sqrt_l_ik")
print("_exp_2h_i, _exp_2h_j, _exp_2l_ij, _exp_2l_ik = np.exp(-2*h_i), np.exp(-2*h_j), np.exp(-2*l_ij), np.exp(-2*l_ik)")
print()
print("cF3 = ", get_ccode(to_norm(cF3(hi, hj, lij, lik).rewrite(sp.exp)).subs(subs_exp)))

kappa = l_ij*h_i - l_ik*h_j
_kappa = 1/kappa
_h_i, _h_j, _l_ij, _l_ik = 1/h_i, 1/h_j, 1/l_ij, 1/l_ik
sqrt_h_i, sqrt_h_j, sqrt_l_ij, sqrt_l_ik = np.sqrt(h_i), np.sqrt(h_j), np.sqrt(l_ij), np.sqrt(l_ik)
_sqrt_h_i, _sqrt_h_j, _sqrt_l_ij, _sqrt_l_ik = 1/sqrt_h_i, 1/sqrt_h_j, 1/sqrt_l_ij, 1/sqrt_l_ik
_exp_2h_i, _exp_2h_j, _exp_2l_ij, _exp_2l_ik = np.exp(-2*h_i), np.exp(-2*h_j), np.exp(-2*l_ij), np.exp(-2*l_ik)

cF3 =  -(_exp_2h_i*_exp_2h_i)*_exp_2h_j*F3((0.5)*M_SQRT2*sqrt_h_i*sqrt_h_j/sqrt_l_ij - M_SQRT2*sqrt_h_i*sqrt_l_ij/sqrt_h_j - 0.5*M_SQRT2*sqrt_h_j*sqrt_l_ij/sqrt_h_i, sqrt_h_i*sqrt_l_ij/(sqrt_h_j*sqrt_l_ik), M_SQRT2*\lambda_{ij}*h_i/(h_j*sqrt_l_ik) - M_SQRT2*sqrt_l_ik) + (_exp_2h_i*_exp_2h_i)*(_exp_2l_ij*_exp_2l_ij)*F3((0.5)*M_SQRT2*sqrt_h_i*sqrt_h_j/sqrt_l_ij - M_SQRT2*sqrt_h_i*sqrt_l_ij/sqrt_h_j + (0.5)*M_SQRT2*sqrt_h_j*sqrt_l_ij/sqrt_h_i, sqrt_h_i*sqrt_l_ij/(sqrt_h_j*sqrt_l_ik), M_SQRT2*\lambda_{ij}*h_i/(h_j*sqrt_l_ik) - M_SQRT2*sqrt_l_ik) + 2.*_exp_2h_i*_exp_2h_j*_exp_2l_ij*_exp_2

In [96]:
def sigmas_args(arg_s1, arg_s2, arg_s3):
    result = ""
    if arg_s1:
        result += "1"
    if arg_s2:
        result += "2"
    if arg_s3:
        result += "3"
    return result


def sigmas_func_name(func):
    name = ""
    if isinstance(func, EFunction):
        name = "E"
    if isinstance(func, FFunction):
        name = "F"
    if isinstance(func, FTildeFunction):
        name = "FTilde"
    return name + sigmas_args(*func.args[-3:])

In [97]:
subs_kappa = {
    lij*hi - lik*hj: 1/sp.Symbol("_kappa")
}

sigmas_funcs = []

def optimize_moment(expr):
    def add_sigmas_func(func):
        if func not in sigmas_funcs:
            sigmas_funcs.append(func)
        return sp.Symbol(sigmas_func_name(func)+"_cF3")*cF3(hi, hj, lij, lik)
    expr = expr.subs(subs_kappa).expand()
    expr = expr.replace(EFunction, lambda *args: add_sigmas_func(EFunction(*args)))
    expr = expr.replace(FFunction, lambda *args: add_sigmas_func(FFunction(*args)))
    expr = expr.replace(FTildeFunction, lambda *args: add_sigmas_func(FTildeFunction(*args)))
    return expr

In [ ]:
print("mi = ", get_ccode(optimize_moment(expr_mi)))
# print("expr_m_i_diff_h_i = ", get_ccode(optimize_moment(expr_m_i.diff(h_i)).subs(subs_coeffs)))
# print("expr_m_i_diff_h_j = ", get_ccode(optimize_moment(expr_m_i.diff(h_j)).subs(subs_coeffs)))
# print("expr_m_i_diff_l_ij = ", get_ccode(optimize_moment(expr_m_i.diff(l_ij)).subs(subs_coeffs)))
# print("expr_m_i_diff_l_ik = ", get_ccode(optimize_moment(expr_m_i.diff(l_ik)).subs(subs_coeffs)))
# print()
# print("expr_m_j = ", get_ccode(optimize_moment(expr_m_j).subs(subs_coeffs)))
# print("expr_m_j_diff_h_i = ", get_ccode(optimize_moment(expr_m_j.diff(h_i)).subs(subs_coeffs)))
# print("expr_m_j_diff_h_j = ", get_ccode(optimize_moment(expr_m_j.diff(h_j)).subs(subs_coeffs)))
# print("expr_m_j_diff_l_ij = ", get_ccode(optimize_moment(expr_m_j.diff(l_ij)).subs(subs_coeffs)))
# print("expr_m_j_diff_l_ik = ", get_ccode(optimize_moment(expr_m_j.diff(l_ik)).subs(subs_coeffs)))
# print()
# print("expr_eta_ij = ", get_ccode(optimize_moment(expr_eta_ij).subs(subs_coeffs)))
# print("expr_eta_ij_diff_h_i = ", get_ccode(optimize_moment(expr_eta_ij.diff(h_i)).subs(subs_coeffs)))
# print("expr_eta_ij_diff_h_j = ", get_ccode(optimize_moment(expr_eta_ij.diff(h_j)).subs(subs_coeffs)))
# print("expr_eta_ij_diff_l_ij = ", get_ccode(optimize_moment(expr_eta_ij.diff(l_ij)).subs(subs_coeffs)))
# print("expr_eta_ij_diff_l_ik = ", get_ccode(optimize_moment(expr_eta_ij.diff(l_ik)).subs(subs_coeffs)))
# print()
# print("expr_eta_ik = ", get_ccode(optimize_moment(expr_eta_ik).subs(subs_coeffs)))
# print("expr_eta_ik_diff_h_i = ", get_ccode(optimize_moment(expr_eta_ik.diff(h_i)).subs(subs_coeffs)))
# print("expr_eta_ik_diff_h_j = ", get_ccode(optimize_moment(expr_eta_ik.diff(h_j)).subs(subs_coeffs)))
# print("expr_eta_ik_diff_l_ij = ", get_ccode(optimize_moment(expr_eta_ik.diff(l_ij)).subs(subs_coeffs)))
# print("expr_eta_ik_diff_l_ik = ", get_ccode(optimize_moment(expr_eta_ik.diff(l_ik)).subs(subs_coeffs)))

expr_m_i =  -0.125*E123_cF3*_kappa*sqrt_h_j*sqrt_l_ij*sqrt_l_ik/sqrt_h_i + (0.25)*M_SQRT2*F13_cF3*\lambda_{ij}*_kappa*sqrt_l_ik - 0.125*M_SQRT2*FTilde123_cF3*\lambda_{ik}*_kappa*h_j*sqrt_h_j/(sqrt_h_i*sqrt_l_ij) + (0.25)*M_SQRT2*FTilde123_cF3*_kappa*sqrt_h_i*sqrt_h_j*sqrt_l_ij + (0.125)*M_SQRT2*FTilde12_cF3*\lambda_{ik}*_kappa*h_j*sqrt_h_j*sqrt_l_ij/(h_i*sqrt_h_i) - 0.25*\lambda_{ij}*\lambda_{ik}*_kappa*(h_j*h_j)/(h_i*h_i) - 0.25*\lambda_{ij}*_kappa + (0.25)*\lambda_{ik}*_kappa*(h_j*h_j)/l_ij - 0.25*_kappa*h_i*h_j + (0.25)*_kappa*h_j*(l_ij*l_ij)/h_i - (0.25)/h_i


In [102]:
for sigmas_func in sigmas_funcs:
    print(f"{sigmas_func_name(sigmas_func)}_cF3 = _cF3*({get_ccode(to_norm(sigmas_func.rewrite(sp.exp)).subs(subs_exp).subs(subs_coeffs))})")

E123_cF3 = _cF3*(-(_exp_2h_i*_exp_2h_i)*_exp_2h_j + (_exp_2h_i*_exp_2h_i)*(_exp_2l_ij*_exp_2l_ij) + 2.*_exp_2h_i*_exp_2h_j*_exp_2l_ij*_exp_2l_ik - 2.*_exp_2h_i*_exp_2l_ij*_exp_2l_ik - _exp_2h_j*(_exp_2l_ij*_exp_2l_ij) + 1.)
F13_cF3 = _cF3*(-(_exp_2h_i*_exp_2h_i)*_exp_2h_j*dawson(-0.5*M_SQRT2*sqrt_h_i*sqrt_h_j/sqrt_l_ij + M_SQRT2*sqrt_h_i*sqrt_l_ij/sqrt_h_j + (0.5)*M_SQRT2*sqrt_h_j*sqrt_l_ij/sqrt_h_i) - (_exp_2h_i*_exp_2h_i)*(_exp_2l_ij*_exp_2l_ij)*dawson((0.5)*M_SQRT2*sqrt_h_i*sqrt_h_j/sqrt_l_ij - M_SQRT2*sqrt_h_i*sqrt_l_ij/sqrt_h_j + (0.5)*M_SQRT2*sqrt_h_j*sqrt_l_ij/sqrt_h_i) - _exp_2h_j*(_exp_2l_ij*_exp_2l_ij)*dawson((0.5)*M_SQRT2*sqrt_h_i*sqrt_h_j/sqrt_l_ij + M_SQRT2*sqrt_h_i*sqrt_l_ij/sqrt_h_j - 0.5*M_SQRT2*sqrt_h_j*sqrt_l_ij/sqrt_h_i) + dawson((0.5)*M_SQRT2*sqrt_h_i*sqrt_h_j/sqrt_l_ij + M_SQRT2*sqrt_h_i*sqrt_l_ij/sqrt_h_j + (0.5)*M_SQRT2*sqrt_h_j*sqrt_l_ij/sqrt_h_i))
FTilde123_cF3 = _cF3*((_exp_2h_i*_exp_2h_i)*_exp_2h_j*dawson(-0.5*M_SQRT2*h_i/sqrt_l_ik + (0.5)*M_SQRT2*l_ij/sqrt_l

In [ ]:
def break_to_sub_exprs(name, expr, max_sub_exprs=60):
    args = expr.args
    N = len(args) // max_sub_exprs
    flag_first = True
    for _ in range(N):
        if flag_first:
            s = f"{name} = "
            flag_first = False
        else:
            s = f"{name} += "
        print(s, get_ccode(sp.Add(*args[:max_sub_exprs])))
        args = args[max_sub_exprs:]

    if len(args) > 0:
        if flag_first:
            s = f"{name} = "
            flag_first = False
        else:
            s = f"{name} += "
        print(s, get_ccode(sp.Add(*args)))

### Вырожденный случай

In [ ]:
sigmas_funcs = []

def optimize_moment_degenerate(expr):
    def add_sigmas_func(func):
        if func not in sigmas_funcs:
            sigmas_funcs.append(func)
        return sp.Symbol(sigmas_func_name(func)+"_cF3")*cF3(hi, hj, lij, lik)
    
    expr = evaluate_cF3_degenerate(expr).subs(subs_to_cF3Function_degenerate).expand()
    expr = expr.replace(cF3_degenerate, lambda *args: cF3(*args[:4]))
    expr = expr.replace(EFunction, lambda *args: add_sigmas_func(EFunction(*args)))
    expr = expr.replace(FFunction, lambda *args: add_sigmas_func(FFunction(*args)))
    return expr

In [ ]:
# print("expr_m_i = ", get_ccode(optimize_moment_degenerate(expr_m_i_degenerate).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_i_diff_h_i", (optimize_moment_degenerate(expr_m_i_degenerate.diff(h_i)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_i_diff_h_j", (optimize_moment_degenerate(expr_m_i_degenerate.diff(h_j)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_i_diff_l_ij", (optimize_moment_degenerate(expr_m_i_degenerate.diff(l_ij)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_i_diff_l_ik", (optimize_moment_degenerate(expr_m_i_degenerate.diff(l_ik)).subs(subs_coeffs)))
# print()
# print("expr_m_j = ", get_ccode(optimize_moment_degenerate(expr_m_j_degenerate).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_j_diff_h_i", (optimize_moment_degenerate(expr_m_j_degenerate.diff(h_i)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_j_diff_h_j", (optimize_moment_degenerate(expr_m_j_degenerate.diff(h_j)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_j_diff_l_ij", (optimize_moment_degenerate(expr_m_j_degenerate.diff(l_ij)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_m_j_diff_l_ik", (optimize_moment_degenerate(expr_m_j_degenerate.diff(l_ik)).subs(subs_coeffs)))
# print()
# print("expr_eta_ij = ", get_ccode(optimize_moment_degenerate(expr_eta_ij_degenerate).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ij_diff_h_i", (optimize_moment_degenerate(expr_eta_ij_degenerate.diff(h_i)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ij_diff_h_j", (optimize_moment_degenerate(expr_eta_ij_degenerate.diff(h_j)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ij_diff_l_ij", (optimize_moment_degenerate(expr_eta_ij_degenerate.diff(l_ij)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ij_diff_l_ik", (optimize_moment_degenerate(expr_eta_ij_degenerate.diff(l_ik)).subs(subs_coeffs)))
# print()
# print("expr_eta_ik = ", get_ccode(optimize_moment_degenerate(expr_eta_ik_degenerate).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ik_diff_h_i", (optimize_moment_degenerate(expr_eta_ik_degenerate.diff(h_i)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ik_diff_h_j", (optimize_moment_degenerate(expr_eta_ik_degenerate.diff(h_j)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ik_diff_l_ij", (optimize_moment_degenerate(expr_eta_ik_degenerate.diff(l_ij)).subs(subs_coeffs)))
# break_to_sub_exprs("expr_eta_ik_diff_l_ik", (optimize_moment_degenerate(expr_eta_ik_degenerate.diff(l_ik)).subs(subs_coeffs)))

In [ ]:
# for sigmas_func in sigmas_funcs:
#     print(f"{sigmas_func_name(sigmas_func)}_cF3 = _cF3*({get_ccode(to_norm(sigmas_func.rewrite(sp.exp)).subs(subs_exp).subs(subs_coeffs))})")

### Вырожденный случай с $h_1 = h_2$ и $\lambda_1 = \lambda_2$

In [ ]:
subs_coeffs_degenerate_spec_1 = {
    h: sp.Symbol("h"),
    l: sp.Symbol("l")
}

subs_exp_degenerate_spec_1 = {
    sp.exp(-2*h): sp.Symbol("_exp_2h"),
    sp.exp(-2*l): sp.Symbol("_exp_2l"),
}

def to_norm_degenerate_spec_1(expr):
    expr *= sp.exp(-3*h-3*l)
    return expr.expand()

In [ ]:
print("_h, _l, = 1/h, 1/l")
print("sqrt_h, sqrt_l, = np.sqrt(h), np.sqrt(l)")
print("_sqrt_h, _sqrt_l, = 1/sqrt_h, 1/sqrt_l")
print("_exp_2h, _exp_2l = np.exp(-2*h), np.exp(-2*l)")
print()
print("cF3 = ", get_ccode(to_norm_degenerate_spec_1(cF3(h, h, l, l).rewrite(sp.exp)).subs(subs_exp_degenerate_spec_1).subs(subs_coeffs_degenerate_spec_1)))
print("_cF3 = 1/cF3")

_h, _l, = 1/h, 1/l
sqrt_h, sqrt_l, = np.sqrt(h), np.sqrt(l)
_sqrt_h, _sqrt_l, = 1/sqrt_h, 1/sqrt_l
_exp_2h, _exp_2l = np.exp(-2*h), np.exp(-2*l)

cF3 =  -(_exp_2h*_exp_2h*_exp_2h)*F3((0.5)*M_SQRT2*_sqrt_l*h - 1.5*M_SQRT2*sqrt_l, 1., 0.) + 3.*(_exp_2h*_exp_2h)*(_exp_2l*_exp_2l)*F3((0.5)*M_SQRT2*_sqrt_l*h - 0.5*M_SQRT2*sqrt_l, 1., 0.) - 3.*_exp_2h*(_exp_2l*_exp_2l)*F3((0.5)*M_SQRT2*_sqrt_l*h + (0.5)*M_SQRT2*sqrt_l, 1., 0.) + F3((0.5)*M_SQRT2*_sqrt_l*h + (1.5)*M_SQRT2*sqrt_l, 1., 0.)
_cF3 = 1/cF3


In [ ]:
sigmas_funcs = []

def optimize_moment_degenerate_spec_1(expr):
    def add_sigmas_func(func):
        if func not in sigmas_funcs:
            sigmas_funcs.append(func)
        return sp.Symbol(sigmas_func_name(func)+"_cF3")*cF3(h, h, l, l)
    
    expr = expr.subs(subs_to_cF3Function_degenerate_spec_1).expand()
    expr = expr.replace(EFunction_spec_1, EFunction).replace(FFunction_spec_1, FFunction)
    expr = expr.replace(cF3_degenerate, lambda *args: cF3(*args[:4]))
    expr = expr.replace(EFunction, lambda *args: add_sigmas_func(EFunction(*args)))
    expr = expr.replace(FFunction, lambda *args: add_sigmas_func(FFunction(*args)))
    return expr

In [ ]:
print("expr_m = ", get_ccode(optimize_moment_degenerate_spec_1(expr_m_i_degenerate_spec_1).subs(subs_coeffs_degenerate_spec_1)))
print("expr_m_diff_h = ", get_ccode(optimize_moment_degenerate_spec_1(expr_m_i_degenerate_spec_1.diff(h)).subs(subs_coeffs_degenerate_spec_1)))
print("expr_m_diff_l = ", get_ccode(optimize_moment_degenerate_spec_1(expr_m_i_degenerate_spec_1.diff(l)).subs(subs_coeffs_degenerate_spec_1)))
print()
print("expr_eta = ", get_ccode(optimize_moment_degenerate_spec_1(expr_eta_ij_degenerate_spec_1).subs(subs_coeffs_degenerate_spec_1)))
print("expr_eta_diff_h = ", get_ccode(optimize_moment_degenerate_spec_1(expr_eta_ij_degenerate_spec_1.diff(h)).subs(subs_coeffs_degenerate_spec_1)))
print("expr_eta_diff_l = ", get_ccode(optimize_moment_degenerate_spec_1(expr_eta_ij_degenerate_spec_1.diff(l)).subs(subs_coeffs_degenerate_spec_1)))

expr_m =  (1.0/6.0)*M_SQRT2*F123_cF3*_sqrt_l - 1.0/3.0*_h - 1.0/3.0*_l*h
expr_m_diff_h =  -1.0/12.0*M_SQRT2*E123_cF3*F123_cF3*_l*_sqrt_l*h + (1.0/6.0)*E123_cF3*_l - 1.0/6.0*M_SQRT2*E12_cF3*F123_cF3*_sqrt_l + (1.0/6.0)*M_SQRT2*E13_cF3*F123_cF3*_sqrt_l + (1.0/6.0)*(F123_cF3*F123_cF3)*(_l*_l)*(h*h) - 1.0/6.0*(F123_cF3*F123_cF3)*_l + (0.5)*F123_cF3*F12_cF3*_l*h - 1.0/6.0*M_SQRT2*F123_cF3*_l*_sqrt_l*h + (1.0/3.0)*(_h*_h) - 1.0/3.0*_l
expr_m_diff_l =  (1.0/24.0)*M_SQRT2*E123_cF3*F123_cF3*(_l*_l)*_sqrt_l*(h*h) - 0.125*M_SQRT2*E123_cF3*F123_cF3*_sqrt_l - 1.0/12.0*E123_cF3*(_l*_l)*h + (1.0/12.0)*M_SQRT2*E12_cF3*F123_cF3*_l*_sqrt_l*h + (1.0/12.0)*E12_cF3*_l - 1.0/12.0*M_SQRT2*E13_cF3*F123_cF3*_l*_sqrt_l*h + (1.0/6.0)*E13_cF3*_l - 1.0/12.0*M_SQRT2*E1_cF3*F123_cF3*_sqrt_l + (1.0/12.0)*M_SQRT2*E3_cF3*F123_cF3*_sqrt_l - 1.0/12.0*(F123_cF3*F123_cF3)*(_l*_l*_l)*(h*h*h) + (1.0/12.0)*(F123_cF3*F123_cF3)*(_l*_l)*h + (0.25)*(F123_cF3*F123_cF3)*_l*h - 0.25*F123_cF3*F12_cF3*(_l*_l)*(h*h) - 0.25*F123_cF3*F12

In [ ]:
for sigmas_func in sigmas_funcs:
    print(f"{sigmas_func_name(sigmas_func)}_cF3 = _cF3*({get_ccode(to_norm_degenerate_spec_1(sigmas_func.rewrite(sp.exp)).subs(subs_exp_degenerate_spec_1).subs(subs_coeffs_degenerate_spec_1))})")

F123_cF3 = _cF3*(-(_exp_2h*_exp_2h*_exp_2h)*Z2.F2((0.5)*M_SQRT2*_sqrt_l*h - 1.5*M_SQRT2*sqrt_l) + 3.*(_exp_2h*_exp_2h)*(_exp_2l*_exp_2l)*Z2.F2((0.5)*M_SQRT2*_sqrt_l*h - 0.5*M_SQRT2*sqrt_l) - 3.*_exp_2h*(_exp_2l*_exp_2l)*Z2.F2((0.5)*M_SQRT2*_sqrt_l*h + (0.5)*M_SQRT2*sqrt_l) + Z2.F2((0.5)*M_SQRT2*_sqrt_l*h + (1.5)*M_SQRT2*sqrt_l))
E123_cF3 = _cF3*(-(_exp_2h*_exp_2h*_exp_2h) + 3.*(_exp_2h*_exp_2h)*(_exp_2l*_exp_2l) - 3.*_exp_2h*(_exp_2l*_exp_2l) + 1.)
E12_cF3 = _cF3*((_exp_2h*_exp_2h*_exp_2h) - (_exp_2h*_exp_2h)*(_exp_2l*_exp_2l) - _exp_2h*(_exp_2l*_exp_2l) + 1.)
E13_cF3 = _cF3*((_exp_2h*_exp_2h*_exp_2h) - (_exp_2h*_exp_2h)*(_exp_2l*_exp_2l) - _exp_2h*(_exp_2l*_exp_2l) + 1.)
F12_cF3 = _cF3*((_exp_2h*_exp_2h*_exp_2h)*Z2.F2((0.5)*M_SQRT2*_sqrt_l*h - 1.5*M_SQRT2*sqrt_l) - (_exp_2h*_exp_2h)*(_exp_2l*_exp_2l)*Z2.F2((0.5)*M_SQRT2*_sqrt_l*h - 0.5*M_SQRT2*sqrt_l) - _exp_2h*(_exp_2l*_exp_2l)*Z2.F2((0.5)*M_SQRT2*_sqrt_l*h + (0.5)*M_SQRT2*sqrt_l) + Z2.F2((0.5)*M_SQRT2*_sqrt_l*h + (1.5)*M_SQRT2*sqrt_

In [ ]:
subs_moments = {
    m: sp.Symbol("m"),
    eta: sp.Symbol("eta"),
}


In [ ]:
expr = expr_m_j_eta_ij_degenerate_spec_1.subs(subs_to_cF3Function_degenerate_spec_1).expand()
expr = expr.replace(EFunction_spec_1, EFunction).replace(FFunction_spec_1, FFunction).subs(subs_moments)
expr = expr.replace(cF3_degenerate, lambda *args: cF3(*args[:4]))
expr = expr.subs(subs_coeffs_degenerate_spec_1)
print(CPrinter().doprint(expr))

-0.1*eta*(h*h*h)/(l*l) + 3.*eta*(h*h)*EFunction_norm(h, h, l, l, 1, 1, 0)/(10.*M_SQRT2*h*sqrt(l)*FFunction_norm(h, h, l, l, 1, 1, 1) + 30.*M_SQRT2*pow(l, 1.5)*FFunction_norm(h, h, l, l, 1, 1, 0) - 20.*l*cF3Function_norm(h, h, l, l)) + (0.7)*eta*h - 3.*eta*h*EFunction_norm(h, h, l, l, 1, 0, 0)/(5.*M_SQRT2*h*FFunction_norm(h, h, l, l, 1, 1, 1)/sqrt(l) + 15.*M_SQRT2*sqrt(l)*FFunction_norm(h, h, l, l, 1, 1, 0) - 10.*cF3Function_norm(h, h, l, l)) - 0.6*eta*h/l - 3.*eta*l*EFunction_norm(h, h, l, l, 1, 0, 0)/(5.*M_SQRT2*(h*h)*FFunction_norm(h, h, l, l, 1, 1, 1)/sqrt(l) + 15.*M_SQRT2*h*sqrt(l)*FFunction_norm(h, h, l, l, 1, 1, 0) - 10.*h*cF3Function_norm(h, h, l, l)) - 9.*eta*l*EFunction_norm(h, h, l, l, 1, 1, 0)/(10.*M_SQRT2*h*FFunction_norm(h, h, l, l, 1, 1, 1)/sqrt(l) + 30.*M_SQRT2*sqrt(l)*FFunction_norm(h, h, l, l, 1, 1, 0) - 20.*cF3Function_norm(h, h, l, l)) + 3.*eta*l*EFunction_norm(h, h, l, l, 0, 0, 0)/(5.*M_SQRT2*h*FFunction_norm(h, h, l, l, 1, 1, 1)/sqrt(l) + 15.*M_SQRT2*sqrt(l)*FFunct

## Решение обратной задачи

### Определения

In [ ]:
import Z3 as Z3_module
import Z2 as Z2_module
import Z1 as Z1_module
import datlib
import numpy as np

In [ ]:
result = Z3_module.find_coeffs_symmetrical(.1, .3, .4)
print(result)

coeffs = result
moments = Z3_module.calc_m_i_symmetrical(*coeffs), Z3_module.calc_eta_ij_symmetrical(*coeffs), Z3_module.calc_eta_ik_symmetrical(*coeffs)
print(moments)
print(Z3_module.calc_m_j_symmetrical(*coeffs))
Z3_module.calc_Q_star_symmetrical(*moments, *coeffs) 

(0.1743748986678629, 0.199340544178533, 0.6954488712579345, 1.1710874741890245)
(0.1000000000000032, 0.30000000000000104, 0.4000000000000026)
0.09999999999999787


-0.8487632127299367

In [ ]:
# Z3_module.find_coeffs_symmetrical_from_Q_star(np.asarray([.1, .1]), np.asarray([.3, .3]), np.asarray([-0.8487632123453477, -0.8487632123453477]))

### Полностью симметричный случай

In [ ]:
data_m = np.linspace(.001, .99, 100)
data_zeta = np.linspace(.001, .99, 100)

data_m, data_zeta = np.meshgrid(data_m, data_zeta)

data_eta = data_zeta + (1-data_zeta)*data_m**2

In [ ]:
data_Z2_h, data_Z2_l = Z2_module.find_coeffs_symmetrical(data_m, data_eta)
data_Z2_upsilon = Z2_module.calc_upsilon_symmetrical(data_Z2_h, data_Z2_l)
data_eta2 = Z2_module.calc_eta2_symmetrical(data_Z2_h, data_Z2_l)
data_Z2_m_par_2 = Z2_module.calc_m_par_2_symmetrical(data_Z2_h, data_Z2_l)

In [ ]:
data_Z3_h, data_Z3_l = Z3_module.find_coeffs_all_symmetrical(data_m, data_eta)
data_Q_star = Z3_module.calc_Q_star_symmetrical(data_m, data_m, data_eta, data_eta, data_Z3_h, data_Z3_h, data_Z3_l, data_Z3_l)
data_Q = data_Q_star - data_eta2 + 1
data_Z3_upsilon = .5*(1-Z3_module.calc_m_j_eta_ij_symmetrical(data_m, data_m, data_eta, data_eta, data_Z3_h, data_Z3_h, data_Z3_l, data_Z3_l)/data_m)
data_Z3_m_par_2 = Z3_module.calc_m_j_par_2_symmetrical(data_m, data_m, data_eta, data_eta, data_Z3_h, data_Z3_h, data_Z3_l, data_Z3_l)

/tmp/ipykernel_1269003/3502303516.py:4: RuntimeWarning: overflow encountered in calc_m_j_eta_ij_symmetrical
  data_Z3_upsilon = .5*(1-Z3_module.calc_m_j_eta_ij_symmetrical(data_m, data_m, data_eta, data_eta, data_Z3_h, data_Z3_h, data_Z3_l, data_Z3_l)/data_m)
/tmp/ipykernel_1269003/3502303516.py:4: RuntimeWarning: invalid value encountered in calc_m_j_eta_ij_symmetrical
  data_Z3_upsilon = .5*(1-Z3_module.calc_m_j_eta_ij_symmetrical(data_m, data_m, data_eta, data_eta, data_Z3_h, data_Z3_h, data_Z3_l, data_Z3_l)/data_m)


In [ ]:
datlib.np2dat(
    "data/Z3_all_symmetrical.dat", 
    "m eta zeta Z2_h Z2_l eta2 Z2_upsilon Z2_m_par_2 Z3_h Z3_l Z3_upsilon Z3_m_par_2 Q_star Q rho_h rho_l".split(), 
    np.asarray([
        data_m, data_eta, data_zeta, data_Z2_h, data_Z2_l, 
        data_eta2, data_Z2_upsilon, data_Z2_m_par_2,
        data_Z3_h, data_Z3_l, data_Z3_upsilon, data_Z3_m_par_2, data_Q_star,
        data_Q, data_Z3_h/data_Z2_h, data_Z3_l/data_Z2_l
    ])
)

### Симметричный случай

In [ ]:
N = 32
left, right = 0, 1

data_m = np.linspace(left, right, N)
data_zeta_1 = np.linspace(left, right, N)
data_zeta_2 = np.linspace(left, right, N)

data_m, data_zeta_1, data_zeta_2 = np.meshgrid(data_m, data_zeta_1, data_zeta_2)

data_zeta_ij = data_zeta_1
data_zeta_ik = data_zeta_2 + (1-data_zeta_2)*data_zeta_ij**2

data_eta_ij = data_zeta_ij + (1-data_zeta_ij)*data_m**2
data_eta_ik = data_zeta_ik + (1-data_zeta_ik)*data_m**2

moments = data_m, data_eta_ij, data_eta_ik

In [ ]:
data_Z2_h_ij, data_Z2_l_ij = Z2_module.find_coeffs_symmetrical(data_m, data_eta_ij)
data_Z2_h_ik, data_Z2_l_ik = Z2_module.find_coeffs_symmetrical(data_m, data_eta_ik)

In [ ]:
data_Z2_upsilon = Z2_module.calc_upsilon_symmetrical(data_Z2_h_ij, data_Z2_l_ij)
data_eta2 = Z2_module.calc_eta2_symmetrical(data_Z2_h_ij, data_Z2_l_ij)

In [ ]:
data_h_i, data_h_j, data_l_ij, data_l_ik = Z3_module.find_coeffs_symmetrical(*moments)
coeffs = data_h_i, data_h_j, data_l_ij, data_l_ik

In [ ]:
data_Q_star = Z3_module.calc_Q_star_symmetrical(*moments, *coeffs)
data_Q = data_Q_star - data_eta2 + 1
data_m_j_eta_ij = Z3_module.calc_m_j_eta_ij_symmetrical(*moments, *coeffs)
# data_m_j_eta_ij[np.abs(data_m_j_eta_ij) > 1] = np.nan
data_Z3_upsilon = .5*(1-data_m_j_eta_ij/data_m)

data_Z3_upsilon[data_Z3_upsilon > .5] = np.nan
data_Z3_upsilon[data_Z3_upsilon < 0] = np.nan
data_Z3_upsilon[np.abs(1-data_Z2_upsilon/data_Z3_upsilon) > .03] = np.nan

In [ ]:
# data_h_i, data_h_j, data_l_ij, data_l_ik = Z3_module.find_coeffs_from_Q_star(data_m, data_eta_ij, data_Q_star)

In [ ]:
def write_frame(file, name, array):  
    file.write(f"SCALARS {name} float 1\n")
    file.write("LOOKUP_TABLE default\n")
    for k in range(array.shape[2]):
        for j in range(array.shape[1]):
            for i in range(array.shape[0]):
                value = array[j, i, k]
                file.write(f"{value}\n")

file = open("data/Z3.vtk", "w")
file.write("# vtk DataFile Version 2.0\n")
file.write("Z3\n")
file.write("ASCII\n")
file.write("DATASET STRUCTURED_POINTS\n")
file.write(f"DIMENSIONS {N} {N} {N}\n")
file.write(f"SPACING {(right-left)/(N-1)} {(right-left)/(N-1)} {(right-left)/(N-1)}\n")
file.write(f"ORIGIN {left} {left} {left}\n")
file.write(f"POINT_DATA {N**3}\n")

# write_frame(file, "m", data_m)
# write_frame(file, "zeta_1", data_zeta_1)
# write_frame(file, "zeta_2", data_zeta_2)

# write_frame(file, "rho_h_i", data_h_i/data_Z2_h_ij)
# write_frame(file, "rho_h_j", data_h_j/data_Z2_h_ik)
# write_frame(file, "rho_l_ij", data_l_ij/data_Z2_l_ij)
# write_frame(file, "rho_l_ik", data_l_ik/data_Z2_l_ik)

write_frame(file, "Q_star", data_Q_star)

write_frame(file, "U_err_abs", np.abs(data_Z2_upsilon-data_Z3_upsilon))
write_frame(file, "U_err_rel", np.abs(1-data_Z2_upsilon/data_Z3_upsilon))


file.close()

In [ ]:
%%bash
vtk2msh data/Z3.vtk

data/Z3.vtk ==> data/Z3.msh ... 
  frame 0: size 32*32*32=32768
  frame 0: step 0.0322581 0.0322581 0.0322581
  frame 0: origin 0 0 0
  frame 0: points 32768
  frame 0: end of header, 14 lines
  frame 0: nan+inf 6180
  frame 1: end of header, 32787 lines
  frame 1: nan+inf 6337
  frame 2: end of header, 65560 lines
  frame 2: nan+inf 6337
  end of file, 98329 lines


### Аппроксимация

In [ ]:
import scipy.optimize as opt

In [ ]:
variables = [data_m, data_zeta_1, data_Q_star]
y_data = data_zeta_2
indexes_nans = np.logical_not(np.isnan(data_Q_star))

def func(_, *p_args, nans=True):
    const = p_args[0]
    linear = np.asarray(p_args[1:4])
    quadratic = np.asarray(p_args[4:13])
    сubic = np.asarray(p_args[13:40])
    
    result = const
    for i in range(3):
        result += linear[i]*variables[i]
    for i in range(3):
        for j in range(3):
            result += quadratic[3*i+j]*variables[i]*variables[j]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                result += сubic[9*i+3*j+k]*variables[i]*variables[j]*variables[k]

    if nans:
        return result[indexes_nans].ravel()
    else:
        return result

In [ ]:
p0 = [1] + 3*[1] + 9*[.01] + 27*[.01]
p = opt.curve_fit(func, None, y_data[indexes_nans].ravel(), p0=p0)[0]
p

array([ 1.85741033e-01,  2.61049533e-01, -5.02074335e-01,  2.05874162e+00,
       -5.59197280e+00, -2.32840732e+02, -1.03313349e+02,  2.35389463e+02,
       -6.21018481e+00, -1.88299184e+02,  1.03799015e+02,  1.84920211e+02,
        3.38181579e+00,  5.62157893e+00, -1.03204561e+02, -5.65746724e+01,
        8.19456249e+02, -3.53487482e+01,  4.85148690e+02, -2.08533762e+02,
        2.46654381e+02, -2.24194635e+02, -7.16669806e+02,  2.52301264e+02,
        1.21563753e+02, -2.17994736e+02,  6.68316701e+00, -9.73810199e+01,
       -5.31331095e+02,  1.62015019e+02,  8.20306428e+01,  2.63307256e+02,
       -1.48640234e+02,  8.57856717e+01, -1.75001728e+02, -6.67379489e+01,
       -1.42026196e+02,  1.38358040e+02,  5.74742364e+01,  9.01883846e-01])

In [ ]:
approx = func(None, *p, nans=False)
approx = np.abs(y_data-approx)
print(np.nanmax(approx)) 
print(np.nanmean(approx))

0.814750832064908
0.08267526162186901


### Равновесные значения из атом-в-атом

In [ ]:
lattice = "VCC3"

names, data = datlib.parse_from_input(open(f"data/Q-{lattice}.dat", "r"))
data_T, data_Hz, data_m, data_eta, data_eta2, data_Q1, data_Q2, data_Q3, data_Q4 = data
data_zeta = (data_eta - data_m**2) / (1 - data_m**2)

In [ ]:
data_p = Z1_module.find_coeffs(data_m)

In [ ]:
data_h, data_l = Z2_module.find_coeffs_symmetrical(data_m, data_eta)
data_upsilon = Z2_module.calc_upsilon_symmetrical(data_h, data_l)
# data_eta2 = Z2_module.calc_eta2_symmetrical(data_h, data_l)

In [ ]:
data_Q_star_1, data_Q_star_2 = data_Q1 + data_eta2 - 1, data_Q2 + data_eta2 - 1
data_Q_star_3, data_Q_star_4 = data_Q3 + data_eta2 - 1, data_Q4 + data_eta2 - 1

In [ ]:
Q1_coeffs = Z3_module.find_coeffs_symmetrical_from_Q_star(data_m, data_eta, data_Q_star_1)
Q2_coeffs = Z3_module.find_coeffs_symmetrical_from_Q_star(data_m, data_eta, data_Q_star_2)
Q3_coeffs = Z3_module.find_coeffs_symmetrical_from_Q_star(data_m, data_eta, data_Q_star_3)
Q4_coeffs = Z3_module.find_coeffs_symmetrical_from_Q_star(data_m, data_eta, data_Q_star_4)

KeyboardInterrupt: 

In [ ]:
names_for_Q1 = "Q1 Q_star_1 Q1_eta_ik Q1_h_i Q1_h_j Q1_l_ij Q1_l_ik"
data_for_Q1 = data_Q1, data_Q_star_1, Z3_module.calc_eta_ik_symmetrical(*Q1_coeffs), *Q1_coeffs

names_for_Q2 = "Q2 Q_star_2 Q2_eta_ik Q2_h_i Q2_h_j Q2_l_ij Q2_l_ik"
data_for_Q2 = data_Q2, data_Q_star_2, Z3_module.calc_eta_ik_symmetrical(*Q2_coeffs), *Q2_coeffs

names_for_Q3 = "Q3 Q_star_3 Q3_eta_ik Q3_h_i Q3_h_j Q3_l_ij Q3_l_ik"
data_for_Q3 = data_Q3, data_Q_star_3, Z3_module.calc_eta_ik_symmetrical(*Q3_coeffs), *Q3_coeffs

names_for_Q4 = "Q4 Q_star_4 Q4_eta_ik Q4_h_i Q4_h_j Q4_l_ij Q4_l_ik"
data_for_Q4 = data_Q4, data_Q_star_4, Z3_module.calc_eta_ik_symmetrical(*Q4_coeffs), *Q4_coeffs

/tmp/ipykernel_3867039/4011780693.py:2: RuntimeWarning: invalid value encountered in calc_eta_ik
  data_for_Q1 = data_Q1, data_Q_star_1, Z3_module.calc_eta_ik(*Q1_coeffs), *Q1_coeffs
/tmp/ipykernel_3867039/4011780693.py:5: RuntimeWarning: invalid value encountered in calc_eta_ik
  data_for_Q2 = data_Q2, data_Q_star_2, Z3_module.calc_eta_ik(*Q2_coeffs), *Q2_coeffs
/tmp/ipykernel_3867039/4011780693.py:8: RuntimeWarning: invalid value encountered in calc_eta_ik
  data_for_Q3 = data_Q3, data_Q_star_3, Z3_module.calc_eta_ik(*Q3_coeffs), *Q3_coeffs
/tmp/ipykernel_3867039/4011780693.py:11: RuntimeWarning: invalid value encountered in calc_eta_ik
  data_for_Q4 = data_Q4, data_Q_star_4, Z3_module.calc_eta_ik(*Q4_coeffs), *Q4_coeffs


In [ ]:
datlib.np2dat(
    f"data/Q-{lattice}-coeffs.dat", 
    f"T Hz m eta zeta eta2 p h l upsilon {names_for_Q1} {names_for_Q2} {names_for_Q3} {names_for_Q4}".split(), 
    np.asarray([
        data_T, data_Hz, data_m, data_eta, data_zeta, data_eta2, 
        data_p, data_h, data_l, data_upsilon,
        *data_for_Q1, *data_for_Q2, *data_for_Q3, *data_for_Q4,
    ])
)

### Ошибка $\Upsilon$

In [ ]:
!ls data/dat-right/SC*-tvals.dat

data/dat-right/SC-T0.1-tvals.dat  data/dat-right/SC-T0.8-tvals.dat
data/dat-right/SC-T0.2-tvals.dat  data/dat-right/SC-T0.9-tvals.dat
data/dat-right/SC-T0.3-tvals.dat  data/dat-right/SC-T1.0-tvals.dat
data/dat-right/SC-T0.4-tvals.dat  data/dat-right/SC-T1.1-tvals.dat
data/dat-right/SC-T0.5-tvals.dat  data/dat-right/SC-T1.2-tvals.dat
data/dat-right/SC-T0.6-tvals.dat  data/dat-right/SC-T1.3-tvals.dat
data/dat-right/SC-T0.7-tvals.dat  data/dat-right/SC-T1.4-tvals.dat


In [ ]:
names, data = datlib.parse_from_input(open("data/dat-right/BCC-T1.1-tvals.dat", "r"))
data_m = data[names.index("M")]
data_eta = data[names.index("eta")]
data_U_LL = data[names.index("UM_LL")]/data_m
data_U_CMD = data[names.index("U_CMD")]
data_eta_k2 = data[names.index("eta_k2")]
data_eta_k3 = data[names.index("eta_k3")]
data_eta_k4 = data[names.index("eta_k4")]

In [ ]:
coeffs = Z2_module.find_coeffs_symmetrical(data_m, data_eta)
data_U_Z2 = Z2_module.calc_upsilon_symmetrical(*coeffs)

In [ ]:
data_eta_mean = np.zeros_like(data_eta)
data_eta_mean[:] = np.mean(data_eta)
coeffs = Z2_module.find_coeffs_symmetrical(data_m, data_eta_mean)
data_U_Z2_mean = Z2_module.calc_upsilon_symmetrical(*coeffs)

In [ ]:
coeffs = Z3_module.find_coeffs_symmetrical(data_m, data_eta, data_eta_k2)
data_U_Z3_k2 = .5*(1-Z3_module.calc_m_j_eta_ij_symmetrical(data_m, data_eta, data_eta_k2, *coeffs)/data_m)

coeffs = Z3_module.find_coeffs_symmetrical(data_m, data_eta, data_eta_k3)
data_U_Z3_k3 = .5*(1-Z3_module.calc_m_j_eta_ij_symmetrical(data_m, data_eta, data_eta_k3, *coeffs)/data_m)

In [ ]:
datlib.np2dat(
    "data/upsilon_error.dat", 
    "M eta U_LL U_CMD U_Z2 U_Z2_mean U_Z3_k2 U_Z3_k3".split(), 
    np.asarray([
        data_m, data_eta, data_U_LL, data_U_CMD,
        data_U_Z2, data_U_Z2_mean,
        data_U_Z3_k2, data_U_Z3_k3
    ])
)

In [ ]:
%%bash
gplt3 -U 'U_LL(M) U_CMD(M) U_Z2(M) U_Z3_k2(M) U_Z3_k3(M)' data/upsilon_error.dat -rx .3: -to data/picts/pict.pdf 
# gplt3 -U 'eta(M)' data/dat-right/SC-T0.1-tvals.dat -to data/picts/pict.pdf 

         line 0: warning: Resetting primary output file to /tmp/gplt3-1698744-9228225-p.tex,
                  PostScript output to /tmp/gplt3-1698744-9228225-p.eps


### Асимптотика при $\left<m\right> \rightarrow 1$ 

In [ ]:
import scipy.optimize as opt

In [ ]:
def find_coeffs_Z3(m, eta_ij, eta_ik):
    if m == 1 or eta_ij == 1 or eta_ik == 1:
        return np.nan, np.nan, np.nan, np.nan
    if m == 0 or eta_ij == 0 or eta_ik == 0:
        return np.nan, np.nan, np.nan, np.nan
    
    def func(x):
        return [
            Z3_module.calc_m_i_symmetrical_single(*x) - m,
            Z3_module.calc_m_j_symmetrical_single(*x) - m,
            Z3_module.calc_eta_ij_symmetrical_single(*x) - eta_ij,
            Z3_module.calc_eta_ik_symmetrical_single(*x) - eta_ik,
        ]

    result = opt.fsolve(func, Z3_module.coeffs_symmetrical_approx(m, eta_ij, eta_ik), full_output=True)
    if result[2] != 1:
        return np.nan, np.nan, np.nan, np.nan
    return result[0]

In [ ]:
find_coeffs_Z3(.5, .01, .01)

(nan, nan, nan, nan)

In [ ]:
# %timeit Z3_module.find_coeffs_symmetrical_single(.1, .3, .4)
# %timeit find_coeffs_Z3(.1, .3, .4)

In [ ]:
def find_coeffs_Z2(m, eta):
    if m == 1 or eta == 1:
        return np.nan, np.nan
    
    def func(x):
        h, l = x
        return [
            Z2_module.calc_m_symmetrical_single(h, l) - m,
            Z2_module.calc_eta_symmetrical_single(h, l) - eta,
        ]

    result = opt.fsolve(func, Z2_module.coeffs_symmetrical_approx(m, eta), full_output=True)
    if result[2] != 1:
        return np.nan, np.nan
    return result[0]

In [ ]:
N = 32

data_m = np.linspace(.99, 1, N)
data_eta_ij = np.linspace(.99, 1, N)
data_eta_ik = np.linspace(.99, 1, N)

data_m, data_eta_ij, data_eta_ik = np.meshgrid(data_m, data_eta_ij, data_eta_ik)

In [ ]:
data_h_i, data_h_j = np.empty_like(data_m), np.empty_like(data_m)
data_l_ij, data_l_ik = np.empty_like(data_m), np.empty_like(data_m)

for i in range(data_m.size):
    result = find_coeffs_Z3(data_m.flat[i], data_eta_ij.flat[i], data_eta_ik.flat[i])
    data_h_i.flat[i] = result[0]
    data_h_j.flat[i] = result[1]
    data_l_ij.flat[i] = result[2]
    data_l_ik.flat[i] = result[3]

In [ ]:
data_Z2_h_ij = np.empty_like(data_m)
data_Z2_l_ij = np.empty_like(data_m)

for i in range(data_m.size):
    result = find_coeffs_Z2(data_m.flat[i], data_eta_ij.flat[i])
    data_Z2_h_ij.flat[i] = result[0]
    data_Z2_l_ij.flat[i] = result[1]


data_Z2_h_ik = np.empty_like(data_m)
data_Z2_l_ik = np.empty_like(data_m)

for i in range(data_m.size):
    result = find_coeffs_Z2(data_m.flat[i], data_eta_ik.flat[i])
    data_Z2_h_ik.flat[i] = result[0]
    data_Z2_l_ik.flat[i] = result[1]

In [ ]:
data_eta2 = Z2_module.calc_eta2_symmetrical(data_Z2_h_ij, data_Z2_l_ij)

In [ ]:
data_Q_star = Z3_module.calc_Q_star_symmetrical(data_m, data_eta_ij, data_eta_ik, data_h_i, data_h_j, data_l_ij, data_l_ik)
data_Q = data_Q_star + 1 - data_eta2

In [ ]:
def write_frame(file, name, array):  
    file.write(f"SCALARS {name} float 1\n")
    file.write("LOOKUP_TABLE default\n")
    for k in range(array.shape[2]):
        for j in range(array.shape[1]):
            for i in range(array.shape[0]):
                value = array[j, i, k]
                file.write(f"{value}\n")

file = open("data/Z3.vtk", "w")
file.write("# vtk DataFile Version 2.0\n")
file.write("Z3\n")
file.write("ASCII\n")
file.write("DATASET STRUCTURED_POINTS\n")
file.write(f"DIMENSIONS {N} {N} {N}\n")
file.write(f"SPACING {.01/(N-1)} {.01/(N-1)} {.01/(N-1)}\n")
file.write(f"ORIGIN .99 .99 .99\n")
file.write(f"POINT_DATA {N**3}\n")

write_frame(file, "h_i", data_h_i)
write_frame(file, "h_j", data_h_j)
write_frame(file, "l_ij", data_l_ij)
write_frame(file, "l_ik", data_l_ik)

write_frame(file, "Q_star", data_Q_star)
write_frame(file, "Q", data_Q)

file.close()

In [ ]:
%%bash
vtk2msh data/Z3.vtk

data/Z3.vtk ==> data/Z3.msh ... 
  frame 0: size 32*32*32=32768
  frame 0: step 0.000322581 0.000322581 0.000322581
  frame 0: origin 0.99 0.99 0.99
  frame 0: points 32768
  frame 0: end of header, 14 lines
  frame 0: nan+inf 20269
  frame 1: end of header, 32787 lines
  frame 1: nan+inf 20269
  frame 2: end of header, 65560 lines
  frame 2: nan+inf 20269
  frame 3: end of header, 98333 lines
  frame 3: nan+inf 20269
  frame 4: end of header, 131106 lines
  frame 4: nan+inf 20269
  frame 5: end of header, 163879 lines
  frame 5: nan+inf 20269
  end of file, 196648 lines


In [ ]:
variables = 1-data_m, 1-data_eta_ij, 1-data_eta_ik
y_data = data_Q
indexes_nans = np.logical_not(np.isnan(data_h_i))

def func(_, *p_args, nans=True):    
    result = p_args[0]
    result += p_args[1]*variables[0] + p_args[2]*variables[1] + p_args[3]*variables[2]

    if nans:
        return result[indexes_nans].ravel()
    else:
        return result
    

p0 = [0] + 3*[1]
p = opt.curve_fit(func, None, y_data[indexes_nans].ravel(), p0=p0)[0]
p

array([-3.82855050e-05,  1.33657070e-04, -1.97983886e+00,  9.94846533e-01])

In [ ]:
# x = 1 - m
# y = 1 - eta_ij
# z = 1 - eta_ik

# Q = -2y + z